##BERT

###Funções Gerais

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install transformers
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=fdda0fad1d78fe9c8da47899e6211a2acbfcdb3b1a1f59b2e97ba41f1f1743bb
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [ ]:
from datetime import datetime

inicio = datetime.now()
print(inicio)

2023-02-16 19:26:28.563845


In [ ]:
#CDJur - Grupos de entidades
import tqdm
import os

def get_data(path_file):
  sentence_id = 0
  data = []
  #for p, _, files in tqdm.tqdm(os.walk(os.path.abspath(path_dataset))):
  #  for file_in in tqdm.tqdm(files):
      #print(file_in)
  #    if file_in == 'train.conll' or file_in == 'test.conll':
  #      continue
  with open(path_file, encoding="utf-8") as f:
    for line in f:
      splits = line.split(' ')

      if len(splits) == 2:
        item = {}
        item['sentence_idx'] = sentence_id
        item['word'] = splits[0]
        item['tag'] = splits[-1].replace('\n', '').strip()
        if (item['word'] == '' or item['tag'] == ''):
          if (item['tag'].startswith('B')):
            print('word: ' + item['word'])
            print('tag: ' + item['tag'])
        else:
            data.append(item)
      #elif len(splits) == 1
      elif line.endswith('.conll\n'): #alterando para considerar um arquivo por completo e não dividi-lo por sentenças; nessa alteração, sentence_id corresponderá ao id do arquivo
        sentence_id += 1

  print(len(data))
  return data

In [ ]:
## Imports
import argparse
#from keras_preprocessing.sequence import pad_sequences
#from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import os
from transformers import BertTokenizer, BertForTokenClassification, BertConfig
from seqeval.metrics import classification_report, f1_score
from sklearn.metrics import confusion_matrix
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange

In [ ]:
import os
import tqdm
import re

# Create class for reading in and separating sentences from their labels
class SentenceGetter(object):
    def __init__(self, data_path, tag2idx):
        self.sents = []
        self.labels = []

        #for p, _, files in tqdm.tqdm(os.walk(os.path.abspath(data_path))):
        #  for file_in in files:
        #    with open(os.path.join(p, file_in), encoding="utf-8") as f:

        with open(data_path) as f:
            if slide_length == 0:
              txt = f.read().split("\n\n")
            else:
              #txt = re.split('.txt', f.read()) #Separa por arquivo
              txt = [t for t in re.split(r'\b[\w\d_]*\.txt', f.read()) if len(t) > 0]

        print('Texto: ' + str(len(txt)))
        self.sents_raw = [(sent.split("\n")) for sent in txt]
        print('Sents_raw: ' + str(len(self.sents_raw)))

        qtde_tokens = 0

        for sent in self.sents_raw:
            tok_lab_pairs = [pair.split() for pair in sent if len(pair.split()) > 1]

            # Handles (very rare) formatting issue causing IndexErrors
            try:
                toks = [pair[0] for pair in tok_lab_pairs]
                labs = [pair[-1] for pair in tok_lab_pairs]

                # In the Russian data, a few invalid labels such as '-' were produced
                # by the spaCy preprocessing. Because of that, we generate a mask to
                # check if there are any invalid labels in the sequence, and if there
                # are, we reindex `toks` and `labs` to exclude them.
                mask = [False if l not in tag2idx else True for l in labs]
                if any(mask):
                    toks = list(np.array(toks)[mask])
                    labs = list(np.array(labs)[mask])

            except IndexError:
                print(sent)
                continue

            if len(toks) > 0:
                self.sents.append(toks)
                self.labels.append(labs)

        print(f"Constructed SentenceGetter with {len(self.sents)} examples.")
        print(f"Labels {len(self.labels)}.")

In [ ]:
def level_entity_new (labels, preds):

  labels_cm, preds_cm = [], []

  for id, lab in enumerate(labels):
    encontrou = False
    #verifica a predição das entidades != de O
    if lab.startswith('B-'):
      encontrou = False
      id_inicio, id_fim = id, id

      #verifica onde termina a entidade
      for id_prox in range(id+1, len(labels)):
        if not labels[id_prox].startswith('I-'):
          id_fim = id_prox - 1
          break

      ent_pred = 'O'
      for id_pred in range(id_inicio, id_fim+1):
        if preds[id_pred][2:] == lab[2:]:
          ent_pred = preds[id_pred]
          encontrou = True
          break
        else:
          ent_pred = preds[id_pred]

      labels_cm.append(lab)
      if ent_pred == 'O':
        preds_cm.append('O')
      else:
        preds_cm.append('B-' + ent_pred[2:])
    elif lab == 'O':
      #verifica se a predição e a label são O ou se a predição é uma parte da label e já foi contada anteriormente
      if preds[id] == lab or encontrou:
        labels_cm.append('O')
        preds_cm.append('O')
      else:
        if preds[id].startswith('B-'):
          encontrou_pred = False
          #verifica até onde termina a entidade se terá uma label real
          for id_prox in range(id+1, len(labels)):
            if preds[id_prox].startswith('I-'):
              if labels[id_prox] != 'O':
                encontrou_pred = True
                break
            else:
              break
          #se não tiver label anotada em toda a entidade predita, adiciona;
          #caso contrário, não adiciona, pois será adicionado pela verificação da label
          if not encontrou_pred:
            labels_cm.append('O')
            preds_cm.append('B-' + preds[id][2:])

  cr = classification_report([labels_cm], [preds_cm])

  header = sorted(list(set(labels_cm + preds_cm)))
  cm = confusion_matrix(labels_cm, preds_cm, labels=header)
  mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(cm)]
  content_cm = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

  return cr, content_cm

In [ ]:
def level_entity (labels, preds):

  labels_cm, preds_cm = [], []
  inicio = 0#, antes, depois = 0, 0, 0
  aux_preds = {}

  def add_pred_entity(aux_preds, label_true):
    tag_pred = None
    if (len(aux_preds) > 1 and label_true[2:] in aux_preds):
      aux_preds.pop(label_true[2:])
      if (len(aux_preds) == 1 and 'O' in aux_preds):
        tag_pred = label_true[2:]
        #antes += 1

    if tag_pred == None:
      tag_pred = max(aux_preds, key = lambda chave: aux_preds[chave])

    if len(tag_pred) > 1:
      tag_pred = 'B-' + tag_pred
    preds_cm.append(tag_pred)
    return 0, {}

  for index in range(0, len(labels)):
    if labels[index] == 'O':
      if inicio == 1:
        inicio, aux_preds = add_pred_entity(aux_preds, labels_cm[len(labels_cm)-1])
      labels_cm.append(labels[index])

      if preds[index].startswith('I'):
        preds_cm.append('O')
        #depois += 1
      else:
        preds_cm.append(preds[index])

    else:
      if labels[index].startswith('B'):
        if inicio == 1:
          inicio, aux_preds = add_pred_entity(aux_preds, labels_cm[len(labels_cm)-1])
        labels_cm.append(labels[index])
        inicio = 1

        tag = preds[index]
        if len(tag) > 1:
          tag = tag[2:]

        if tag in aux_preds:
          aux_preds[tag] += 1
        else:
          aux_preds[tag] = 1

      elif labels[index].startswith('I'):
        tag = preds[index]
        if len(tag) > 1:
          tag = tag[2:]

        if tag in aux_preds:
          aux_preds[tag] += 1
        else:
          aux_preds[tag] = 1

  if inicio == 1:
    inicio, aux_preds = add_pred_entity(aux_preds, labels_cm[len(labels_cm)-1])

  #print(labels_cm)
  #print(preds_cm)

  cr = classification_report([labels_cm], [preds_cm])

  header = sorted(list(set(labels_cm + preds_cm)))
  cm = confusion_matrix(labels_cm, preds_cm, labels=header)
  mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(cm)]
  content_cm = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

  #print(cr)
  #print(content_cm)

  #print('Antes: ' + str(antes))
  #print('Depois: ' + str(depois))

  return cr, content_cm

In [ ]:
import os

par_vitima, par_reu, par_autor, par_testemunha = 0, 0, 0, 0

class BertDataset:
    def __init__(self, sg, tokenizer, max_len, slide_length, tag2idx):

        """
        Takes care of the tokenization and ID-conversion steps
        for prepping data for BERT.

        Takes a SentenceGetter (sg) initialized on the data you
        want to use as argument.
        """

        pad_tok = tokenizer.vocab["[PAD]"]
        sep_tok = tokenizer.vocab["[SEP]"]
        o_lab = tag2idx["O"]

        if slide_length == 0:
          #Tokenize the text into subwords in a label-preserving way
          tokenized_texts = [
              tokenize_and_preserve_labels(sent, labs, tokenizer, 0, 0)
              for sent, labs in zip(sg.sents, sg.labels)
          ]
          self.toks = [["[CLS]"] + text[0] for text in tokenized_texts]
          self.labs = [["O"] + text[1] for text in tokenized_texts]
        else:
          toks_texts = []
          labs_texts = []
          max_context_texts = []
          for sent, labs in zip(sg.sents, sg.labels):
            tokens, labels, max_context = tokenize_and_preserve_labels(sent, labs, tokenizer, max_len, slide_length)
            for t in tokens:
              toks_texts.append(t)
            for l in labels:
              labs_texts.append(l)
            for m in max_context:
              max_context_texts.append(m)

          self.toks = [["[CLS]"] + text for text in toks_texts]
          self.labs = [["O"] + text for text in labs_texts]
          self.max_context = [[0] + text for text in max_context_texts]

        #print('Vítima:', par_vitima)
        #print('Reu:', par_reu)
        #print('Autor:', par_autor)
        #print('Testemunha:', par_testemunha)

        # Convert tokens to IDs
        print('Quantidade de sentenças (tokens):', len(self.toks))
        self.input_ids = pad_sequences(
            [tokenizer.convert_tokens_to_ids(txt) for txt in self.toks],
            maxlen=max_len,
            padding="post",
            dtype="long",
            truncating="post",
        )

        # Convert tags to IDs
        print('Quantidade de sentenças (labels):', len(self.labs))
        tags_ids = [[tag2idx[l] for l in lab] for lab in self.labs]
        self.tags = pad_sequences(
            tags_ids,
            maxlen=max_len,
            value=tag2idx["O"],
            padding="post",
            dtype="long",
            truncating="post",
        )

        if slide_length > 0:
          self.max_context = pad_sequences(
              self.max_context, maxlen=max_len,
              value=0,
              padding="post",
              dtype="long",
              truncating="post",
          )

        # Swaps out the final token-label pair for ([SEP], O)
        # for any sequences that reach the MAX_LEN
        id = -1
        for voc_ids, tag_ids in zip(self.input_ids, self.tags):
            id += 1
            if voc_ids[-1] == pad_tok:
                continue
            else:
                voc_ids[-1] = sep_tok
                tag_ids[-1] = o_lab
                if slide_length > 0:
                  self.max_context[id][-1] = -1

        # Place a mask (zero) over the padding tokens
        self.attn_masks = [[float(i > 0) for i in ii] for ii in self.input_ids]

def tokenize_and_preserve_labels(sentence, text_labels, tokenizer, max_len, slide_length):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    #print('sentence:', len(sentence))

    #print('text_labels:', text_labels)
    for word, label in zip(sentence, text_labels):

      # Tokenize the word and count # of subwords the word is broken into
      tokenized_word = tokenizer.tokenize(word)
      n_subwords = len(tokenized_word)

      # Add the tokenized word to the final tokenized word list
      tokenized_sentence.extend(tokenized_word)

      # Add the same label to the new list of labels `n_subwords` times
      if label.startswith('B-'):
        labels.extend([label])
        if ('I-' + label[2:] not in tag2idx):
          add_tag('I-' + label[2:])
        labels.extend(['I-' + label[2:]] * (n_subwords-1))
      else:
        labels.extend([label] * n_subwords)
      #labels.extend([label])
      #if ('X' not in tag2idx):
      #    add_tag('X')
      #labels.extend(['X'] * (n_subwords-1))

    if slide_length == 0:
      return tokenized_sentence, labels

    n = max_len - 1 #conta menos um token: [CLS]
    tokenized_limite_sentenca = []
    labels_limite_sentenca = []
    list_max_context = []
    for i in range(0, len(tokenized_sentence), n-slide_length):
        final_sentenca = i+n
        if final_sentenca > len(tokenized_sentence):
            final_sentenca = len(tokenized_sentence)
        tokenized_limite_sentenca.append(tokenized_sentence[i:final_sentenca])
        labels_limite_sentenca.append(labels[i:final_sentenca])
        if i == 0:
            t1 = n-(slide_length//2)
            if t1 > final_sentenca:
                t1 = final_sentenca
            t2 = final_sentenca - t1
            max_context = [True] * t1 + [False] * t2
        elif final_sentenca >= len(tokenized_sentence):
            t2 = final_sentenca-i-(slide_length//2)
            max_context = [False] * (slide_length//2) + [True] * t2
        else:
            max_context = [False] * (slide_length//2) + [True] * (n-slide_length) + [False] * (slide_length//2)

        list_max_context.append(max_context)
        if final_sentenca <= n-slide_length//2:
            break

    #for idx, token_sent in enumerate(tokenized_limite_sentenca):
    #  for idx_t, token in enumerate(token_sent):
    #    print(idx_t, token, labels_limite_sentenca[idx][idx_t], list_max_context[idx][idx_t])
    #  print('troca de sentença')
    #  if idx == 2:
    #      break
    #print('tokenized_limite_sentenca:', tokenized_limite_sentenca)
    #print('labels_limite_sentenca:', labels_limite_sentenca)
    #print('list_max_context:', list_max_context)

    #print(len(labels_limite_sentenca))
    for id1, labs_sent in enumerate(labels_limite_sentenca):
      enderecosEncontrados = []
      pessoasEncontradas = []
      for id2, label in enumerate(labs_sent):
        if label.startswith('B-END') and list_max_context[id1][id2]:
          #print('encontrou endereço:', label)
          enderecosEncontrados.append(label)
        if label.startswith('B-PES'):
          #print('encontrou pessoa:', label)
          pessoasEncontradas.append(label)

      '''
      if 'B-PES_VITIMA' in pessoasEncontradas and 'B-END_VITIMA' in enderecosEncontrados:
          print('encontrou vítima:')
          print('labs:', labs_sent)
          print('toks:', tokenized_limite_sentenca[id1])
          global par_vitima
          par_vitima += 1
      if 'B-PES_REU' in pessoasEncontradas and 'B-END_REU' in enderecosEncontrados:
          global par_reu
          par_reu += 1
      if 'B-PES_AUTOR' in pessoasEncontradas and 'B-END_AUTOR' in enderecosEncontrados:
          global par_autor
          par_autor += 1
      if 'B-PES_TESTEMUNHA' in pessoasEncontradas and 'B-END_TESTEMUNHA' in enderecosEncontrados:
          global par_testemunha
          par_testemunha += 1
      '''
    return tokenized_limite_sentenca, labels_limite_sentenca, list_max_context

def flat_accuracy(valid_tags, pred_tags):
    return (np.array(valid_tags) == np.array(pred_tags)).mean()


def annot_confusion_matrix(valid_tags, pred_tags):
    # Create header from unique tags
    header = sorted(list(set(valid_tags + pred_tags)))

    # Calculate the actual confusion matrix
    matrix = confusion_matrix(valid_tags, pred_tags, labels=header)

    # Final formatting touches for the string output
    mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(matrix)]
    content = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

    return content


def get_special_tokens(tokenizer, tag2idx):

    pad_tok = tokenizer.vocab["[PAD]"]
    sep_tok = tokenizer.vocab["[SEP]"]
    cls_tok = tokenizer.vocab["[CLS]"]
    o_lab = tag2idx["O"]

    return pad_tok, sep_tok, cls_tok, o_lab


def load_and_prepare_data(train_data_path, dev_data_path, tokenizer, max_len, slide_length, batch_size, tag2idx):
    getter_train = SentenceGetter(train_data_path, tag2idx)
    getter_dev = SentenceGetter(dev_data_path, tag2idx)

    global par_vitima, par_reu, par_autor, par_testemunha
    par_vitima, par_reu, par_autor, par_testemunha = 0, 0, 0, 0
    train = BertDataset(getter_train, tokenizer, max_len, slide_length, tag2idx)
    par_vitima, par_reu, par_autor, par_testemunha = 0, 0, 0, 0
    dev = BertDataset(getter_dev, tokenizer, max_len, slide_length, tag2idx)

    # Input IDs (tokens), tags (label IDs), attention masks
    tr_inputs = torch.tensor(train.input_ids)
    val_inputs = torch.tensor(dev.input_ids)
    print('tr', len(tr_inputs))
    print('val', len(val_inputs))

    tr_tags = torch.tensor(train.tags)
    val_tags = torch.tensor(dev.tags)
    tr_masks = torch.tensor(train.attn_masks)
    val_masks = torch.tensor(dev.attn_masks)
    if slide_length > 0:
      tr_context = torch.tensor(train.max_context)
      val_context = torch.tensor(dev.max_context)
      train_data = TensorDataset(tr_inputs, tr_masks, tr_tags, tr_context)
      valid_data = TensorDataset(val_inputs, val_masks, val_tags, val_context)
    else:
      train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
      valid_data = TensorDataset(val_inputs, val_masks, val_tags)

    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    valid_sampler = SequentialSampler(valid_data)
    valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

    return train_dataloader, valid_dataloader

def matrix_confusion_class(labels, preds):
      labels_true_cm, labels_pred_cm = [], []
      for l in labels:
        labels_true_cm.extend(l)

      for l in preds:
        labels_pred_cm.extend(l)

      header = sorted(list(set(labels_true_cm + labels_pred_cm)))
      matrix = confusion_matrix(labels_true_cm, labels_pred_cm, labels=header)
      #print(header)
      #print(matrix)
      return str(header) + '\n' + str(matrix)

def cria_arquivos_log(epoch):
  path_arquivo_log = path_geral + "/output_sem_slide_window/log_legalbertpt_epoch" + str(epoch) + ".txt"
  path_arquivo_log2 = path_geral + "/output_sem_slide_window/log2_legalbertpt_epoch" + str(epoch) + ".txt"
  path_arquivo_log3 = path_geral + "/output_sem_slide_window/log3_legalbertpt_epoch" + str(epoch) + ".txt"
  path_arquivo_log4 = path_geral + "/output_sem_slide_window/log4_legalbertpt_epoch" + str(epoch) + ".txt"
  path_arquivo_log5 = path_geral + "/output_sem_slide_window/log5_legalbertpt_epoch" + str(epoch) + ".txt"
  path_arquivo_log2_treino = path_geral + "/output_sem_slide_window/log2_treino_legalbertpt_epoch" + str(epoch) + ".txt"

  if os.path.isfile(path_arquivo_log):
    os.remove(path_arquivo_log)
  arquivo_log = open(path_arquivo_log, "a")

  if os.path.isfile(path_arquivo_log2):
    os.remove(path_arquivo_log2)
  arquivo_log2 = open(path_arquivo_log2, "a")

  if os.path.isfile(path_arquivo_log2_treino):
    os.remove(path_arquivo_log2_treino)
  arquivo_log2_treino = open(path_arquivo_log2_treino, "a")

  if os.path.isfile(path_arquivo_log3):
    os.remove(path_arquivo_log3)
  arquivo_log3 = open(path_arquivo_log3, "a")

  if os.path.isfile(path_arquivo_log4):
    os.remove(path_arquivo_log4)
  arquivo_log4 = open(path_arquivo_log4, "a")

  if os.path.isfile(path_arquivo_log5):
    os.remove(path_arquivo_log5)
  arquivo_log5 = open(path_arquivo_log5, "a")

  return arquivo_log, arquivo_log2, arquivo_log2_treino, arquivo_log3, arquivo_log4, arquivo_log5

def ajustaPredsTokenInicial(pred_tags, wordpieces):
  i = 0
  for token, pred in list(zip(wordpieces, pred_tags)):
    if token.startswith('##') and pred.startswith('I-') and \
       i > 0 and not wordpieces[i-1].startswith('##') and pred_tags[i-1] == 'O':
       pred_tags[i-1] = 'B-' + pred[2:]
    i += 1
  return pred_tags

def armazenaLog(arquivo_log, str_rep, toplevel_labels, toplevel_preds):
  #Armazena no arquivo de log
  lines = []
  lines.append('-----------------------------\n')
  lines.append(str(len(str_rep)) + ' ' + ' ' + str(len(toplevel_labels)) + ' ' + str(len(toplevel_preds)) + '\n')

  for index in range(len(toplevel_labels)):
    try:
      erro = ''
      if toplevel_labels[index] != toplevel_preds[index]:
        erro = '*'
      lines.append(str(str_rep[index]) + '\t' + str(toplevel_labels[index])  + '\t' + str(toplevel_preds[index]) + '\t' + str(erro) + '\n')
    except Exception as e:
      print(e)

  arquivo_log.writelines(lines)

def train_and_save_model(
    model,
    tokenizer,
    optimizer,
    verbose,
    epochs,
    idx2tag,
    tag2idx,
    path_output_model,
    device,
    train_dataloader,
    valid_dataloader,
):
    pad_tok, sep_tok, cls_tok, o_lab = get_special_tokens(tokenizer, tag2idx)
    verbose = verbose
    epochs = epochs

    epoch = 0
    model.to(device)

    melhor_f1, melhor_epoca = 0, 0

    for _ in trange(epochs, desc="Epoch"):

        epoch += 1

        log, log2, log2_treino, log3, log4, log5 = cria_arquivos_log(epoch)

        # Training loop
        print("Starting training loop.")
        model.train()
        tr_loss, tr_accuracy = 0, 0
        nb_tr_examples, nb_tr_steps = 0, 0
        tr_preds, tr_labels = [], []

        #progress_bar = tqdm.tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        #for batch in progress_bar:
        ids_treino = []
        for step, batch in enumerate(train_dataloader):

            #try:
            # Add batch to gpu
            batch = tuple(t.to(device) for t in batch)
            if slide_length == 0:
              b_input_ids, b_input_mask, b_labels = batch
            else:
              b_input_ids, b_input_mask, b_labels, b_context = batch

            # Forward pass
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
            loss, tr_logits = outputs[:2]

            # Backward pass
            loss.backward()

            # Compute train loss
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            if slide_length == 0:
              preds_mask = ((b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok))
            else:
              preds_mask = ((b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok) & (b_context == 1))
            tr_logits = tr_logits.detach().cpu().numpy()
            tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1))

            if len(tr_logits) == 1:
              tr_batch_preds = np.argmax(tr_logits[0][preds_mask.cpu().squeeze()], axis=1)
            else:
              tr_batch_preds = np.argmax(tr_logits[preds_mask.cpu().squeeze()], axis=1)

            #tr_batch_preds = np.argmax(tr_logits[preds_mask.cpu().squeeze()], axis=1)
            tr_batch_labels = tr_label_ids.cpu().numpy()
            tr_preds.extend(tr_batch_preds)
            tr_labels.extend(tr_batch_labels)
            aux_ids_treino = torch.masked_select(b_input_ids, (preds_mask == 1))
            ids_treino.extend(aux_ids_treino.cpu().numpy())

            # Compute training accuracy
            tmp_tr_accuracy = flat_accuracy(tr_batch_labels, tr_batch_preds)
            tr_accuracy += tmp_tr_accuracy

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)

            # Update parameters
            optimizer.step()
            scheduler.step()
            model.zero_grad()

        tr_loss = tr_loss / nb_tr_steps
        tr_accuracy = tr_accuracy / nb_tr_steps

        # Print training loss and accuracy per epoch
        print(f"Train loss: {tr_loss}")
        print(f"Train accuracy: {tr_accuracy}")

        # Report metrics
        # Evaluate loss, acc, conf. matrix, and class. report on devset
        pred_tags = [[idx2tag[i] for i in tr_preds]]
        valid_tags = [[idx2tag[i] for i in tr_labels]]

        #Concatena os tokens gerados pelo WordPiece
        wordpieces = tokenizer.convert_ids_to_tokens(ids_treino)

        #Resultado com ajuste de transição inválida de entidade
        preds = pred_tags[0].copy()
        wp_preds = list(zip(wordpieces, preds))
        wp_labels = list(zip(wordpieces, valid_tags[0].copy()))

        str_rep = " ".join([t[0] for t in wp_preds]).replace(" ##", "").split()
        toplevel_preds = [pair[1] for pair in wp_preds if "##" not in pair[0]]
        toplevel_labels = [pair[1] for pair in wp_labels if "##" not in pair[0]]

        toplevel_preds = ajusta_transicao_entidades_new(toplevel_preds, wordpieces)

        #Armazena no arquivo de log
        armazenaLog(log2_treino, str_rep, toplevel_labels, toplevel_preds)

        cl_report_toplevel = classification_report([toplevel_labels], [toplevel_preds])
        log2_treino.writelines(cl_report_toplevel)
        log2_treino.writelines(str(matrix_confusion_class([toplevel_labels], [toplevel_preds])))

        #Exibe resultado com ajuste de transição a nível de entidade
        cr_level_entity, cm_level_entity = level_entity_new(toplevel_labels, toplevel_preds)
        log2_treino.writelines(cr_level_entity)
        log2_treino.writelines(str(cm_level_entity))

        # Validation loop
        print("Starting validation loop.")

        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels = [], []
        predictions_contexto, true_labels_contexto = [], []

        ids = []
        ids_contexto = []

        for batch in valid_dataloader:

            #try:
            batch = tuple(t.to(device) for t in batch)
            if slide_length == 0:
              b_input_ids, b_input_mask, b_labels = batch
            else:
              b_input_ids, b_input_mask, b_labels, b_context = batch

            with torch.no_grad():
                outputs = model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                )
                tmp_eval_loss, logits = outputs[:2]

            #print('ids', b_input_ids)

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            if slide_length == 0:
              preds_mask = ((b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok))
            else:
              preds_mask = ((b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok) & (b_context == 1))
              preds_mask_contexto = ((b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok))

            logits = logits.detach().cpu().numpy()
            label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            label_ids_contexto = torch.masked_select(b_labels, (preds_mask_contexto == 1))
            #print('logits:', logits)
            if len(logits) == 1:
              val_batch_preds = np.argmax(logits[0][preds_mask.cpu().squeeze()], axis=1)
              val_batch_preds_contexto = np.argmax(logits[0][preds_mask_contexto.cpu().squeeze()], axis=1)
            else:
              val_batch_preds = np.argmax(logits[preds_mask.cpu().squeeze()], axis=1)
              val_batch_preds_contexto = np.argmax(logits[preds_mask_contexto.cpu().squeeze()], axis=1)

            val_batch_labels = label_ids.cpu().numpy()
            val_batch_labels_contexto = label_ids_contexto.cpu().numpy()
            #print('val_batch_preds:', val_batch_preds)
            #val_batch_labels = label_ids.to("cpu").numpy()
            predictions.extend(val_batch_preds)
            true_labels.extend(val_batch_labels)

            predictions_contexto.extend(val_batch_preds_contexto)
            true_labels_contexto.extend(val_batch_labels_contexto)

            aux_ids = torch.masked_select(b_input_ids, (preds_mask == 1))
            ids.extend(aux_ids.cpu().numpy())

            ids_contexto.extend(torch.masked_select(b_input_ids, (preds_mask_contexto == 1)).cpu().numpy())

            exibe_dados(b_input_ids, preds_mask, b_labels, val_batch_labels, val_batch_preds, log)

            tmp_eval_accuracy = flat_accuracy(val_batch_labels, val_batch_preds)

            eval_loss += tmp_eval_loss.mean().item()
            eval_accuracy += tmp_eval_accuracy

            nb_eval_examples += b_input_ids.size(0)

            nb_eval_steps += 1

        # Evaluate loss, acc, conf. matrix, and class. report on devset
        pred_tags = [[idx2tag[i] for i in predictions]]
        valid_tags = [[idx2tag[i] for i in true_labels]]

        #Concatena os tokens gerados pelo WordPiece
        wordpieces = tokenizer.convert_ids_to_tokens(ids)

        #pred_tags = ajustaPredsTokenInicial(pred_tags, wordpieces)

        #Métricas com as predições originais
        cl_report = classification_report(valid_tags, pred_tags)
        log.writelines(cl_report)
        log.writelines(str(matrix_confusion_class(valid_tags, pred_tags)))

        eval_loss = eval_loss / nb_eval_steps
        eval_accuracy = eval_accuracy / nb_eval_steps

        #print(f"Validation loss: {eval_loss}")
        #print(f"Validation Accuracy: {eval_accuracy}")
        #print(f"Classification Report:\n {cl_report}")
        #print(matrix_confusion_class(valid_tags, pred_tags))

        #Armazena log sem retirar a janela de contexto
        wordpieces_contexto = tokenizer.convert_ids_to_tokens(ids_contexto)
        pred_tags_contexto = [[idx2tag[i] for i in predictions_contexto]]
        valid_tags_contexto = [[idx2tag[i] for i in true_labels_contexto]]

        wp_preds_contexto = list(zip(wordpieces_contexto, pred_tags_contexto))
        wp_labels_contexto = list(zip(wordpieces_contexto, valid_tags_contexto[0].copy()))

        #str_rep = " ".join([t[0] for t in wp_preds_contexto]).replace(" ##", "").split()
        #toplevel_preds = [pair[1] for pair in wp_preds_contexto if "##" not in pair[0]]
        #toplevel_labels = [pair[1] for pair in wp_labels_contexto if "##" not in pair[0]]
        str_rep = [t[0] for t in wp_preds_contexto]
        toplevel_preds = [pair[1] for pair in wp_preds_contexto]
        toplevel_labels = [pair[1] for pair in wp_labels_contexto]
        print('log5:', len(wordpieces_contexto), len(toplevel_labels), len(toplevel_preds[0]))
        armazenaLog(log5, wordpieces_contexto, toplevel_labels, toplevel_preds[0])

        #Métricas com as predições retirando os subtokens que iniciam com ##
        preds = pred_tags[0].copy()
        wp_preds = list(zip(wordpieces, preds))
        wp_labels = list(zip(wordpieces, valid_tags[0].copy()))

        str_rep = " ".join([t[0] for t in wp_preds]).replace(" ##", "").split()
        toplevel_preds = [pair[1] for pair in wp_preds if "##" not in pair[0]]
        toplevel_labels = [pair[1] for pair in wp_labels if "##" not in pair[0]]

        cl_report_toplevel = classification_report([toplevel_labels], [toplevel_preds])
        #print(f"Classification Report Top Level:\n {cl_report_toplevel}")
        #print(matrix_confusion_class([toplevel_labels], [toplevel_preds]))

        #Resultado a nível de entidade sem transição de entidade
        cr_level_entity, cm_level_entity = level_entity_new(toplevel_labels, toplevel_preds)
        #print(f"Classification Report Level Entity (Sem transição de entidades):\n {cr_level_entity}")
        #print(cm_level_entity)

        #Resultado com ajuste de transição inválida de entidade
        preds = pred_tags[0].copy()
        wp_preds = list(zip(wordpieces, preds))
        wp_labels = list(zip(wordpieces, valid_tags[0].copy()))

        str_rep = " ".join([t[0] for t in wp_preds]).replace(" ##", "").split()
        toplevel_preds = [pair[1] for pair in wp_preds if "##" not in pair[0]]
        toplevel_labels = [pair[1] for pair in wp_labels if "##" not in pair[0]]

        armazenaLog(log3, str_rep, toplevel_labels, toplevel_preds)

        toplevel_preds = ajusta_transicao_entidades_new(toplevel_preds, wordpieces)

        #Exibe resultado com ajuste de transição a nível de token
        print('-----------------------------------------------')
        print('Ajuste de transição inválida de entidade')
        print('----- COMPARAÇÃO EXATA -----')
        f1_macro = f1_score([toplevel_labels], [toplevel_preds], average='macro')
        if f1_macro > melhor_f1:
          melhor_f1 = f1_macro
          melhor_epoca = epoch

        cl_report_toplevel = classification_report([toplevel_labels], [toplevel_preds])
        print(f"Classification Report Top Level (Ajuste de transição):\n {cl_report_toplevel}")
        print(matrix_confusion_class([toplevel_labels], [toplevel_preds]))

        #Armazena no arquivo de log
        armazenaLog(log2, str_rep, toplevel_labels, toplevel_preds)
        log2.writelines(cl_report_toplevel)
        log2.writelines(str(matrix_confusion_class([toplevel_labels], [toplevel_preds])))

        #Exibe resultado com ajuste de transição a nível de entidade
        cr_level_entity, cm_level_entity = level_entity_new(toplevel_labels, toplevel_preds)
        log2.writelines(cr_level_entity)
        log2.writelines(str(cm_level_entity))
        print('----- COMPARAÇÃO PARCIAL A NÍVEL DE ENTIDADE -----')
        print(f"Classification Report Level Entity:\n {cr_level_entity}")
        print(cm_level_entity)

        #Resultado com ajuste de transição inválida de entidade para O
        preds = pred_tags[0].copy()
        wp_preds = list(zip(wordpieces, preds))
        wp_labels = list(zip(wordpieces, valid_tags[0].copy()))

        str_rep = " ".join([t[0] for t in wp_preds]).replace(" ##", "").split()
        toplevel_preds = [pair[1] for pair in wp_preds if "##" not in pair[0]]
        toplevel_labels = [pair[1] for pair in wp_labels if "##" not in pair[0]]

        toplevel_preds = ajusta_transicao_entidades_O(toplevel_preds, wordpieces)

        armazenaLog(log4, str_rep, toplevel_labels, toplevel_preds)

        #outros tipos de métrica: exact partial
        #print('STRICT:')
        #print('preds:', wp_preds)
        #calculateMetrics(wp_preds, wp_labels, 'strict')

        #print('TYPE:')
        #calculateMetrics(wp_preds, wp_labels, 'type')

        #Exibe resultado a nível de entidade
        print('-----------------------------------------------')
        print('Ajuste de transição inválida de entidade para O')
        print('----- COMPARAÇÃO EXATA -----')
        cl_report_toplevel = classification_report([toplevel_labels], [toplevel_preds])
        print(f"Classification Report Top Level (Transição de entidades O):\n {cl_report_toplevel}")
        print(matrix_confusion_class([toplevel_labels], [toplevel_preds]))

        print('----- COMPARAÇÃO PARCIAL A NÍVEL DE ENTIDADE -----')
        cr_level_entity, cm_level_entity = level_entity_new(toplevel_labels, toplevel_preds)
        print(f"Classification Report Level Entity (Transição de entidades O):\n {cr_level_entity}")
        print(cm_level_entity)
        log4.writelines(cr_level_entity)
        log4.writelines(str(cm_level_entity))
        #if verbose:
        #print(confusion_matrix(valid_tags, pred_tags))#, labels=header))
        #print(f"Confusion Matrix:\n {conf_mat}")

        #matrix_confusion_class(valid_tags, pred_tags)

        # Save model and optimizer state_dict following every epoch
        #torch.save(model.state_dict(), path_output_model + '/fine_tuning_lener/' + f'/finetuned_BERT_epoch_{epoch}.tar')
        torch.save(model.state_dict(), f'{path_geral}/output_sem_slide_window/fine_tuning_ner/finetuned_legalbertpt_epoch_{epoch}.tar')
        #torch.save(model.state_dict(), f'/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_{entidade}/output_sem_slide_window/fine_tuning_ner/finetuned_legalbertpt_epoch_{epoch}.tar')
        #torch.save(model.state_dict(), '/content/drive/MyDrive/Unifor/NER/cdjur/output_ner/fine_tuning_ner/' + entidade + f'finetuned_legalbertpt_epoch_{epoch}.tar')
        #torch.save(model.state_dict(), '/content/drive/MyDrive/Unifor/NER/cdjur/output_ner/fine_tuning_ner/' + entidade + f'finetuned_legalbertpt_epoch_{epoch}.tar')
        '''
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "train_loss": tr_loss,
                "train_acc": tr_accuracy,
                "eval_loss": eval_loss,
                "eval_acc": eval_accuracy,
                "classification_report": cl_report,
                "confusion_matrix": conf_mat,
            },
            ,
            model
            save_path,
        )
        '''
    print(f'Melhor f1:{melhor_f1}\nMelhor época:{melhor_epoca}')

In [ ]:
from operator import itemgetter

def ajusta_transicao_entidades_new(toplevel_preds, str_rep):

  for id, pred in enumerate(toplevel_preds):
    if pred.startswith('I-'):
      #Troca para B- se a primeira entidade iniciar com I-
      if id == 0:
        toplevel_preds[id] = 'B-' + pred[2:]

      #Transição O I
      if toplevel_preds[id-1] == 'O':
          encontrouAnterior = False
          #print(id-1, id-3)
          for id_ant in range(id-1, id-5, -1): #Janela: 3
            if id_ant == -1:
              break
            if toplevel_preds[id_ant] != 'O':
              if toplevel_preds[id_ant][2:] == pred[2:]:
                for i in range(id_ant+1, id-1+1):
                  toplevel_preds[i] = 'I-' + pred[2:]
                encontrouAnterior = True
                break
          if not encontrouAnterior:
            toplevel_preds[id] = 'B-' + pred[2:]

      #Transição I-A I-B ou B-A I-B
      elif toplevel_preds[id-1][2:] != pred[2:]: #ant inicia com B- ou I- e entidades são diferentes
        #verifica se a categoria das entidades ant e atual são diferentes
        if toplevel_preds[id-1][2:5] != pred[2:5]:
          toplevel_preds[id] = 'B-' + pred[2:]
        else:
          id_comeco, id_termina = 0, len(toplevel_preds)-1
          #obtém o id onde começa a entidade
          for id_ant in range(id-1, 0, -1):
            if toplevel_preds[id_ant].startswith('B-'):
              id_comeco = id_ant
              break

          #obtém o id onde termina a entidade
          for id_prox in range(id+1, len(toplevel_preds), 1):
            if not toplevel_preds[id_prox].startswith('I-'):
              id_termina = id_prox-1
              break

          #Obtém a entidade de maior ocorrência
          ents = [toplevel_preds[i][2:] for i in range(id_comeco, id_termina+1, 1)]
          d = {}
          for e in ents:
            d[e] = d.get(e, 0) + 1
          d = sorted(d, key = d.get, reverse=True) #ordena a qtde de ocorrências de forma decrescente
          ent_selecionada = d[0] #obtém a entidade que mais ocorre

          #ajusta as entidades
          toplevel_preds[id_comeco] = 'B-' + ent_selecionada
          for i in range(id_comeco+1, id_termina+1):
            toplevel_preds[i] = 'I-' + ent_selecionada



    #Verifica se uma entidade (por completo) está associada a um único caracter ou se for uma Norma está associada a uma única palavra
    if pred.startswith('B') and \
      (id == len(toplevel_preds)-1 or toplevel_preds[id+1] == 'O' or toplevel_preds[id+1].startswith('B-')) and \
      len(str_rep[id]) == 1:
      toplevel_preds[id] = 'O'
  return toplevel_preds

In [ ]:
from operator import itemgetter

def ajusta_transicao_entidades(toplevel_preds, str_rep):

  for id, pred in enumerate(toplevel_preds):
     if pred.startswith('I-'):
      if id == 0:
        toplevel_preds[id] = 'B-' + pred[2:]

      #Transição I-A I-B ou B-A I-B
      if not toplevel_preds[id-1] == 'O' and toplevel_preds[id-1][2:] != pred[2:]:
        if id+1 > len(toplevel_preds) and (not toplevel_preds[id+1].startswith('I-') or \
           (toplevel_preds[id+1].startswith('I-') and toplevel_preds[id+1][2:] == toplevel_preds[id-1][2:])):
          toplevel_preds[id] = 'I-' + toplevel_preds[id-1][2:]
        elif toplevel_preds[id+1].startswith('I-') and toplevel_preds[id+1] == pred:
          toplevel_preds[id] = 'B-' + pred[2:]

      #Transição O I
      if toplevel_preds[id-1] == 'O':
          encontrouAnterior = False
          #print(id-1, id-3)
          for id_ant in range(id-1, id-5, -1): #Janela: 3
            if id_ant == -1:
              break
            if toplevel_preds[id_ant] != 'O':
              if toplevel_preds[id_ant][2:] == pred[2:]:
                for i in range(id_ant+1, id-1+1):
                  toplevel_preds[i] = 'I-' + pred[2:]
                encontrouAnterior = True
                break
          if not encontrouAnterior:
            toplevel_preds[id] = 'B-' + pred[2:]

     #Verifica se uma entidade (por completo) está associada a um único caracter ou se for uma Norma está associada a uma única palavra
     #print(pred)
     #print(len(pred))
     #print(pred)
     #print(pred.startswith('B'))
     #print(id == len(toplevel_preds)-1)
     #print(toplevel_preds[id+1] == 'O')
     #print(toplevel_preds[id+1].startswith('B-'))
     #print(len(str_rep[id]) <= 2)
     #print(pred[2:].startswith('NOR'))
     if pred.startswith('B') and \
      (id == len(toplevel_preds)-1 or toplevel_preds[id+1] == 'O' or toplevel_preds[id+1].startswith('B-')) and \
      len(str_rep[id]) == 1:
      #(len(str_rep[id]) <= 2 or pred[2:].startswith('NOR')):
      #(len(str_rep[id]) == 1 or pred[2:].startswith('NOR')):# str_rep[id].lower() == 'artigo'):
      toplevel_preds[id] = 'O'
  return toplevel_preds

In [ ]:
from operator import itemgetter

def ajusta_transicao_entidades_O(toplevel_preds, str_rep):

  for id, pred in enumerate(toplevel_preds):
     if pred.startswith('I-'):
      if id == 0:
        toplevel_preds[id] = 'O'

      #Transição I-A I-B ou B-A I-B
      if not toplevel_preds[id-1] == 'O' and toplevel_preds[id-1][2:] != pred[2:]:
        if not toplevel_preds[id+1].startswith('I-') or \
           (toplevel_preds[id+1].startswith('I-') and toplevel_preds[id+1][2:] == toplevel_preds[id-1][2:]):
          toplevel_preds[id] = 'O'
        elif toplevel_preds[id+1].startswith('I-') and toplevel_preds[id+1] == pred:
          toplevel_preds[id] = 'O'

      #Transição O I
      if toplevel_preds[id-1] == 'O':
          toplevel_preds[id] = 'O'

     #Verifica se uma entidade (por completo) está associada a um único caracter
     #print(pred)
     #print(len(pred))
     if pred.startswith('B') and \
      (id == len(toplevel_preds)-1 or toplevel_preds[id+1] == 'O' or toplevel_preds[id+1].startswith('B-')) and \
      len(str_rep[id]) == 1:
      #(len(str_rep[id]) <= 2 or pred[2:].startswith('NOR')):
      toplevel_preds[id] = 'O'
  return toplevel_preds

In [ ]:
def exibe_dados(b_input_ids, preds_mask, b_labels, tr_labels, tr_preds, arquivo):

  aux = []
  for index_sentence in range(len(b_input_ids)):
    for index_token in range(len(b_input_ids[index_sentence])):
      if preds_mask[index_sentence][index_token]:
        aux.append(b_input_ids[index_sentence][index_token])

  ids = tokenizer.convert_ids_to_tokens(aux)
  labels_tag = [idx2tag[l] for l in tr_labels]
  preds_tag = [idx2tag[l] for l in tr_preds]

  lines = []
  lines.append('-----------------------------\n')
  lines.append(str(len(labels_tag)) + ' ' + ' ' + str(len(ids)) + ' ' + str(len(preds_tag)) + '\n')
  for index in range(len(labels_tag)):
    erro = ''
    if labels_tag[index] != preds_tag[index]:
      erro = '*'
    lines.append(ids[index] + '\t' + labels_tag[index]  + '\t' + preds_tag[index] + '\t' + erro + '\n')

  arquivo.writelines(lines)
  #print(classification_report([labels_tag], [preds_tag]))
  #print(matrix_confusion_class([labels_tag], [preds_tag]))

###Treino

In [ ]:
#CDJur
entidade = 'exp1'

# path_cdjur_train = '/content/drive/MyDrive/Unifor/NER/cdjur_' + entidade + '/treino_' + entidade + '.conll'
# path_cdjur_test = '/content/drive/MyDrive/Unifor/NER/cdjur_' + entidade + '/teste_' + entidade + '.conll'
#path_geral = '/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_210922/' + entidade


path_cdjur_train = '/content/drive/MyDrive/UNIFOR/NER/CDJur/lener/train.txt'
path_cdjur_val = '/content/drive/MyDrive/UNIFOR/NER/CDJur/lener/dev.txt'
path_cdjur_test = '/content/drive/MyDrive/UNIFOR/NER/CDJur/lener/test.txt'
#path_cdjur_train = path_geral + '/treino.conll'
#path_cdjur_val = path_geral + '/val.conll'
#path_cdjur_test = path_geral + '/teste.conll'
#path_cdjur_train = '/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_090822/lener_train_ents_cdjur_lener.conll'
#path_cdjur_val = '/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_090822/lener_dev_ents_cdjur_lener.conll'

import pandas as pd
print('treino')
df_data_train = pd.DataFrame(get_data(path_cdjur_train))
print('validação')
df_data_val = pd.DataFrame(get_data(path_cdjur_val))
print('teste')
df_data_test = pd.DataFrame(get_data(path_cdjur_test))

treino
1325555
validação
381229
teste
355227


In [ ]:
df_data_test

,sentence_idx,word,tag
0,0,PODER,O
1,0,JUDICIÁRIO,O
2,0,DO,O
3,0,ESTADO,O
4,0,DO,O
...,...,...,...
355222,0,estes,O
355223,0,"autos,",O
355224,0,Direto.,O
355225,0,cretaria,O


In [ ]:
tags_vals = list(set(df_data_train["tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {idx: tags_vals[idx] for idx in range(0, len(tags_vals))}

In [ ]:
def add_tag(tag):
  indice = len(tag2idx)
  tag2idx[tag] = indice
  idx2tag[indice] = tag

In [ ]:
len(tag2idx)

43

In [ ]:
tag2idx

{'I-NOR_JURISPRUDÊNCIA': 0,
 'B-PES_AUTORID_POLICIAL': 1,
 'B-SENTENÇA': 2,
 'B-END_AUTOR': 3,
 'I-PES_ADVOG': 4,
 'I-END_TESTEMUNHA': 5,
 'B-PES_VITIMA': 6,
 'B-PES_JUIZ': 7,
 'I-PES_JUIZ': 8,
 'I-END_OUTROS': 9,
 'I-PROVA': 10,
 'I-PES_AUTOR': 11,
 'I-PES_TESTEMUNHA': 12,
 'I-PES_PROMOTOR_MP': 13,
 'B-PES_REU': 14,
 'B-END_TESTEMUNHA': 15,
 'O': 16,
 'B-END_DELITO': 17,
 'I-NOR_PRINCIPAL': 18,
 'B-PROVA': 19,
 'I-END_AUTOR': 20,
 'I-END_VITIMA': 21,
 'B-NOR_ACESSORIA': 22,
 'B-NOR_JURISPRUDÊNCIA': 23,
 'B-PES_TESTEMUNHA': 24,
 'I-END_REU': 25,
 'B-END_VITIMA': 26,
 'I-PENA': 27,
 'I-SENTENÇA': 28,
 'I-END_DELITO': 29,
 'I-PES_VITIMA': 30,
 'I-NOR_ACESSORIA': 31,
 'B-PES_ADVOG': 32,
 'B-PENA': 33,
 'B-PES_AUTOR': 34,
 'I-PES_AUTORID_POLICIAL': 35,
 'B-END_OUTROS': 36,
 'I-PES_OUTROS': 37,
 'B-NOR_PRINCIPAL': 38,
 'B-PES_PROMOTOR_MP': 39,
 'I-PES_REU': 40,
 'B-END_REU': 41,
 'B-PES_OUTROS': 42}

In [ ]:
#CDJur
from transformers import BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

#path_cdjur = '/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_' + entidade + '/'
train_data_path = path_cdjur_train #path_cdjur + 'treino.conll' #'/content/drive/MyDrive/Unifor/NER/lener/train_atualizado.txt' #path_cdjur + '/train/labeled/train.conll'
dev_data_path = path_cdjur_test #path_cdjur + '/teste.conll'
max_len = 512
slide_length = 128 #0
batch_size = 8
epochs = 10

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')#'cuda'

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

In [ ]:
train_dataloader, valid_dataloader = load_and_prepare_data(train_data_path, dev_data_path, tokenizer, max_len, slide_length, batch_size, tag2idx)

Texto: 681
Sents_raw: 681
Constructed SentenceGetter with 681 examples.
Labels 681.
Texto: 170
Sents_raw: 170
Constructed SentenceGetter with 170 examples.
Labels 170.
Quantidade de sentenças (tokens): 6395
Quantidade de sentenças (labels): 6395
Quantidade de sentenças (tokens): 1677
Quantidade de sentenças (labels): 1677
tr 6395
val 1677


In [ ]:

model = BertForTokenClassification.from_pretrained('neuralmind/bert-base-portuguese-cased',
                                                   num_labels=len(tag2idx))

optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8, correct_bias=False)

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * epochs
)
verbose = True

Some weights of the model checkpoint at /content/drive/MyDrive/UNIFOR/LegalBert-pt/model-scratch-v4_p1/ were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpo

In [ ]:
device = 'cuda'

#Colab Checkpoint 40k
path_output_model = ''
train_and_save_model(
    model,
    tokenizer,
    optimizer,
    verbose,
    epochs,
    idx2tag,
    tag2idx,
    path_output_model,
    device,
    train_dataloader,
    valid_dataloader
)

teste2


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Starting training loop.
Train loss: 0.4932535109319724
Train accuracy: 0.8925398414611617
Starting validation loop.


/usr/local/lib/python3.8/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


log5: 801817 801817 801817


...

Classification Report Top Level (Transição de entidades O):
                       precision    recall  f1-score   support

           END_AUTOR       0.00      0.00      0.00        18
          END_DELITO       0.17      0.07      0.09        61
          END_OUTROS       0.00      0.00      0.00        81
             END_REU       0.10      0.12      0.11       152
      END_TESTEMUNHA       0.27      0.04      0.08        68
          END_VITIMA       0.00      0.00      0.00        27
       NOR_ACESSORIA       0.28      0.28      0.28       990
  NOR_JURISPRUDÊNCIA       0.21      0.08      0.12       333
       NOR_PRINCIPAL       0.19      0.27      0.23       791
                PENA       0.00      0.00      0.00        82
           PES_ADVOG       0.25      0.01      0.02       122
           PES_AUTOR       0.12      0.04      0.06       169
PES_AUTORID_POLICIAL       0.40      0.24      0.30       300
            PES_JUIZ       0.44      0.05      0.09        83
        

/usr/local/lib/python3.8/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report Level Entity (Transição de entidades O):
                       precision    recall  f1-score   support

           END_AUTOR       0.00      0.00      0.00        18
          END_DELITO       0.75      0.20      0.31        61
          END_OUTROS       0.00      0.00      0.00        81
             END_REU       0.57      0.43      0.49       152
      END_TESTEMUNHA       0.70      0.10      0.18        68
          END_VITIMA       0.00      0.00      0.00        27
       NOR_ACESSORIA       0.85      0.74      0.79       990
  NOR_JURISPRUDÊNCIA       0.83      0.32      0.46       333
       NOR_PRINCIPAL       0.73      0.79      0.76       791
                PENA       0.00      0.00      0.00        82
           PES_ADVOG       0.67      0.02      0.03       122
           PES_AUTOR       0.51      0.15      0.23       169
PES_AUTORID_POLICIAL       0.85      0.43      0.57       300
            PES_JUIZ       0.89      0.10      0.17        83
     

Epoch:  60%|██████    | 6/10 [1:25:44<57:05, 856.32s/it]  

Starting training loop.


# SpaCy

In [ ]:
pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 563 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=4cd6be2ccb7145bf3e7a40b1761b1cce01e16278bc0fd2f96d723ec758ca1e2c
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import spacy
from spacy.lang.pt.examples import sentences
import os
import json
from pathlib import Path
import pickle
import random
from spacy.util import minibatch, compounding
from spacy.training import Example
import tqdm
import re
from nltk import word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

from seqeval.metrics import classification_report
import numpy as np
import pandas as pd

In [ ]:
data_revisao = '131022'

model_train = '/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_spacy/model/'
path_train = f'/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_{data_revisao}/exp1/treino.conll'
path_teste = f'/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_{data_revisao}/exp1/teste.conll'
path_train_saida = f'/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_{data_revisao}/spacy/examples_train.txt'
path_teste_saida = f'/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_{data_revisao}/spacy/teste_output.txt'

In [ ]:
!python -m spacy download pt_core_news_sm
import spacy

2022-10-14 09:44:57.941711: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 463 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
def get_train_data(test, input_train_file, output_train_file):
    train_data, labels_cdjur, entities = [], [], []
    start, end, index = 0, 0, 0
    text = ''

    file_reader = open(input_train_file)
    label = ''
    for line in tqdm.tqdm(file_reader.readlines()):
        #quebra de linha
        if line.endswith('.txt\n'): #if (line == '\n'):#len(tokens) < 2 or line == '\n':
            if label != '':
                entities.append((start, end, label))

            #Verifica se na sentença tem entidades nomeadas
            #if test == 1 or len(entities) > 0:
            train_data.append((text, {"entities" : entities}))

            #reinicializa variaveis
            entities = []
            text, label = '', ''
            index, start, end = 0, 0, 0
        else:
            tokens = line.split()
            if len(tokens) == 2:
                text_token = tokens[0]
                label_token = tokens[-1]

                text += text_token + ' '

                #Verifica se finalizou a entidade
                if (label_token == 'O' or label_token.startswith('B')):
                    if label != '':
                      entities.append((start, end, label))
                      if label not in labels_cdjur:
                          labels_cdjur.append(label)
                    label = ''
                    start = 0
                    end = 0

                if label_token.startswith('B'):
                    start = index
                    end = index + len(text_token)
                    label = label_token[2:]

                if label_token.startswith('I'):
                    end = index + len(text_token)

                index = len(text)

            if len(tokens) > 2:
                print('error')

    if label != '':
      entities.append((start, end, label))

    if len(entities) > 0:
      #Verifica se na sentença tem entidades nomeadas
      #if test == 1 or len(entities) > 0:
      train_data.append((text, {"entities" : entities}))

    with open(output_train_file, 'wb') as fp:
        pickle.dump(train_data, fp)

    print('Salvou os dados de treinamnto!')
    return train_data, labels_cdjur

In [ ]:
#Train the spaCy with new data
#model = model used to load spaCy (None = considers model blank)
#n_iter = number iterations
#train_data = data with new entities
#output_dir = diretory to save the model
def train(model, n_iter, train_data, output_dir, labels_cdjur):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("pt")  # create blank Language class
        print("Created blank 'pt' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe("ner", last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # Add new entity labels to entity recognizer
    for i in labels_cdjur:
        ner.add_label(i)
    move_names = list(ner.move_names)

    # add labels
    #for _,annotations in train_data:
    #    for ent in annotations.get("entities"):
    #        ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    #pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    #other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    # only train NER
    with nlp.disable_pipes(*other_pipes):
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            optimizer = nlp.begin_training()
            #optimizer = nlp.create_optimizer()
        else:
            #optimizer = nlp.initialize()
            optimizer = nlp.resume_training()

        for i in range(n_iter):
            random.shuffle(train_data)
            losses = {}
            batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
            for batch in tqdm.tqdm(batches):
                examples = []
                for text, annots in batch:
                    examples.append(Example.from_dict(nlp.make_doc(text), annots))
                losses = nlp.update(examples, sgd=optimizer)
            print("Losses", losses)

    # test the trained model
    #for text, _ in train_data:
        #doc = nlp(text)
        #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [ ]:
def run_train():
  model_spacy_pt = "pt_core_news_sm"

  train_data, labels_cdjur = get_train_data(0, path_train, path_train_saida)
  with open (path_train_saida, 'rb') as fp:
      train_data = pickle.load(fp)

  print(len(train_data))

  train(model_spacy_pt, 20, train_data, model_train, labels_cdjur)

In [ ]:
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix

def level_entity_new (labels, preds):

  labels_cm, preds_cm = [], []

  for id, lab in enumerate(labels):
    encontrou = False
    #verifica a predição das entidades != de O
    if lab.startswith('B-'):
      encontrou = False
      id_inicio, id_fim = id, id

      #verifica onde termina a entidade
      for id_prox in range(id+1, len(labels)):
        if not labels[id_prox].startswith('I-'):
          id_fim = id_prox - 1
          break

      ent_pred = 'O'
      for id_pred in range(id_inicio, id_fim+1):
        if preds[id_pred][2:] == lab[2:]:
          ent_pred = preds[id_pred]
          encontrou = True
          break
        else:
          ent_pred = preds[id_pred]

      labels_cm.append(lab)
      if ent_pred == 'O':
        preds_cm.append('O')
      else:
        preds_cm.append('B-' + ent_pred[2:])
    elif lab == 'O':
      #verifica se a predição e a label são O ou se a predição é uma parte da label e já foi contada anteriormente
      if preds[id] == lab or encontrou:
        labels_cm.append('O')
        preds_cm.append('O')
      else:
        if preds[id].startswith('B-'):
          encontrou_pred = False
          #verifica até onde termina a entidade se terá uma label real
          for id_prox in range(id+1, len(labels)):
            if preds[id_prox].startswith('I-'):
              if labels[id_prox] != 'O':
                encontrou_pred = True
                break
            else:
              break
          #se não tiver label anotada em toda a entidade predita, adiciona;
          #caso contrário, não adiciona, pois será adicionado pela verificação da label
          if not encontrou_pred:
            labels_cm.append('O')
            preds_cm.append('B-' + preds[id][2:])

  cr = classification_report([labels_cm], [preds_cm])

  header = sorted(list(set(labels_cm + preds_cm)))
  cm = confusion_matrix(labels_cm, preds_cm, labels=header)
  mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(cm)]
  content_cm = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

  return cr, content_cm

def level_entity (labels, preds):

  labels_cm, preds_cm = [], []
  inicio = 0#, antes, depois = 0, 0, 0
  aux_preds = {}

  def add_pred_entity(aux_preds, label_true):
    tag_pred = None
    if (len(aux_preds) > 1 and label_true[2:] in aux_preds):
      aux_preds.pop(label_true[2:])
      if (len(aux_preds) == 1 and 'O' in aux_preds):
        tag_pred = label_true[2:]
        #antes += 1

    if tag_pred == None:
      tag_pred = max(aux_preds, key = lambda chave: aux_preds[chave])

    if len(tag_pred) > 1:
      tag_pred = 'B-' + tag_pred
    preds_cm.append(tag_pred)
    return 0, {}

  for index in range(0, len(labels)):
    if labels[index] == 'O':
      if inicio == 1:
        inicio, aux_preds = add_pred_entity(aux_preds, labels_cm[len(labels_cm)-1])
      labels_cm.append(labels[index])

      if preds[index].startswith('I'):
        preds_cm.append('O')
        #depois += 1
      else:
        preds_cm.append(preds[index])

    else:
      if labels[index].startswith('B'):
        if inicio == 1:
          inicio, aux_preds = add_pred_entity(aux_preds, labels_cm[len(labels_cm)-1])
        labels_cm.append(labels[index])
        inicio = 1

        tag = preds[index]
        if len(tag) > 1:
          tag = tag[2:]

        if tag in aux_preds:
          aux_preds[tag] += 1
        else:
          aux_preds[tag] = 1

      elif labels[index].startswith('I'):
        tag = preds[index]
        if len(tag) > 1:
          tag = tag[2:]

        if tag in aux_preds:
          aux_preds[tag] += 1
        else:
          aux_preds[tag] = 1

  if inicio == 1:
    inicio, aux_preds = add_pred_entity(aux_preds, labels_cm[len(labels_cm)-1])

  #print(labels_cm)
  #print(preds_cm)

  cr = classification_report([labels_cm], [preds_cm])

  header = sorted(list(set(labels_cm + preds_cm)))
  cm = confusion_matrix(labels_cm, preds_cm, labels=header)
  mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(cm)]
  content_cm = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

  #print(cr)
  #print(content_cm)

  #print('Antes: ' + str(antes))
  #print('Depois: ' + str(depois))

  return cr, content_cm

def run_test():
    nlp = spacy.load(model_train)

    # Carrega configurações básicas
    punkt_param = PunktParameters()
    with open("/content/drive/MyDrive/UNIFOR/NER/abbrev_list.pkl", "rb") as fp:
        abbrev_list = pickle.load(fp)

    punkt_param.abbrev_types = set(abbrev_list)
    tokenizer = PunktSentenceTokenizer(punkt_param)

    labels_true = []
    labels_predicted = []

    with open(path_teste, 'r', errors="surrogateescape") as txt_file:
        data = {}
        data['ner'] = []
        sentence = ''
        label_true_sentence = []
        tokens_true_sentence = []
        linhas = txt_file.readlines()
        id_linha = -1
        for linha in linhas:
            id_linha += 1
            if linha.endswith('.txt\n') or id_linha == len(linhas)-1: #Condição que representa a finalização da sentença
                doc = nlp(sentence)
                entities_file = []
                for ent in doc.ents:
                    entities = {}
                    entities['start'] = ent.start_char
                    entities['end'] = ent.end_char
                    entities['label'] = ent.label_
                    entities['entity'] = ent.text
                    entities_file.append(entities)

                tokens = sentence.split() #word_tokenize(sentence, language='portuguese')
                label_predicted_sentence = []
                idx_start, idx_end, idx_token = 0, 0, 0
                for t in tokens:
                    idx_end += len(t)
                    label_predicted_sentence.append(check_entity(entities_file, idx_start, idx_end))
                    idx_end += 1 #Aumenta 1 por conta do espaço
                    idx_start = idx_end

                    if t != tokens_true_sentence[idx_token]:
                        print('verificar')

                    #print(t + ' -> ' + label_predicted_sentence[idx_token] + ' -> ' + tokens_true_sentence[idx_token] + ' -> ' + label_true_sentence[idx_token])
                    idx_token += 1

                data['ner'].append({'sentence': sentence, 'label_true': label_true_sentence, 'label_predicted': label_predicted_sentence, 'entities': entities_file})
                labels_true.append(label_true_sentence)
                labels_predicted.append(label_predicted_sentence)

                sentence = ''
                label_true_sentence = []
                tokens_true_sentence = []

            else:
                if linha == '\n':
                  continue
                sentence += linha.split()[0].strip() + ' '
                tokens_true_sentence.append(linha.split()[0].strip())
                label_true_sentence.append(linha.split()[-1])

    with open(path_teste_saida, 'w', encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False)

    print('Avaliação exata:')
    print(classification_report(labels_true, labels_predicted))

    print('Avaliação parcial:')
    labels_, predicted_ = [], []
    for l in labels_true:
      labels_.extend(l)

    for l in labels_predicted:
      predicted_.extend(l)

    cr, content_cm = level_entity_new(labels_, predicted_)
    print(cr)
    print(content_cm)

    return labels_true, labels_predicted

def check_entity(entities, start, end):
    for e in entities:
        if start >= e['start'] and end <= e['end']:
            if start == e['start']:
                return 'B-' + e['label']
            else:
                return 'I-' + e['label']
    return 'O'

def calculate_metrics():

    predictions, true_labels = [], []
    arquivo_labeled = open(path_teste).readlines()
    id_linha = 0
    with open(path_teste_saida, "r") as json_file:
        dados = json.load(json_file)
        ner = dados['ner']
        id_token = 0
        for item in ner:
            text = item['sentence']
            entities = item['entities']

            tokens = word_tokenize(text, language='portuguese')#text.split()
            predict_labels = []
            idx_start = 0
            idx_end = 0

            for t in tokens:
                while arquivo_labeled[id_linha] == '\n': #or t != arquivo_labeled[id_linha].split()[0]:
                    id_linha += 1

                if t != arquivo_labeled[id_linha].split()[0]:
                    print('verificar')
                    if t == '.' or t == ',':
                        continue
                    if t == 's/n-':
                        t = 's/n'

                idx_end += len(t)
                label_predicted = check_entity(entities, idx_start, idx_end)
                label_true = arquivo_labeled[id_linha].split()[-1]
                predict_labels.append(label_predicted)
                true_labels.append(label_true)
                idx_end += 1 #Aumenta 1 por conta do espaço
                idx_start = idx_end

                print(t + ' -> ' + label_predicted + ' -> texto ' + arquivo_labeled[id_linha].split()[0] + ' -> ' + label_true + ' ' + str(id_linha))
                id_token += 1
                id_linha += 1
            #id_linha += 1
    print(classification_report(label_true, predict_labels))
            #arquivo_ner = pd.read_json(os.path.join(p, file)) #open(file).read()
            #texto = arquivo_ner['text']
            #entities = arquivo_ner['entities']
            #print(file)

def check_train():

    arquivo = open(path_train)
    #753
    #1763
    #881
    #851
    q = 0
    for linha in arquivo.readlines():
    #print(linha)
        if (linha == '\n'):
            q += 1
    print(q)

In [ ]:
run_train()

100%|██████████| 809031/809031 [00:01<00:00, 600541.93it/s]


Salvou os dados de treinamnto!
479
Loaded model 'pt_core_news_sm'


120it [02:27,  1.23s/it]


Losses {'ner': 194.17085361602494}


120it [02:23,  1.19s/it]


...

Losses {'ner': 92.86686859592555}


120it [02:22,  1.19s/it]


Losses {'ner': 181.3635348661337}


120it [02:22,  1.18s/it]


Losses {'ner': 29.124611595515262}
Saved model to /content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_spacy/model


In [ ]:
labels_true, labels_predicted = run_test()
#calculate_metrics()

Avaliação exata:
                      precision    recall  f1-score   support

           END_AUTOR       0.03      0.03      0.03        40
          END_DELITO       0.37      0.24      0.29       156
          END_OUTROS       0.00      0.00      0.00       153
             END_REU       0.29      0.50      0.37       283
      END_TESTEMUNHA       0.56      0.12      0.20       120
          END_VITIMA       0.00      0.00      0.00        69
       NOR_ACESSORIA       0.61      0.66      0.63      2365
  NOR_JURISPRUDÊNCIA       0.43      0.42      0.42       753
       NOR_PRINCIPAL       0.65      0.58      0.61      2747
                PENA       0.38      0.37      0.38       139
           PES_ADVOG       0.07      0.13      0.09       228
           PES_AUTOR       0.45      0.42      0.43       481
PES_AUTORID_POLICIAL       0.69      0.24      0.36       822
            PES_JUIZ       0.61      0.30      0.40       183
          PES_OUTROS       0.41      0.26      0.32 

# LSTM + CRF

In [ ]:
'''
Code modified by the authors of leNER-Br:
    Line split on space instead of tab
    Add preprocessing step of replacing digits with zeros
    Add function create_tag_dict
'''

import numpy as np
import os
import re


# shared global variables to be imported from model also
UNK = "$UNK$"
NUM = "$NUM$"
NONE = "O"


# special error message
class MyIOError(Exception):
    def __init__(self, filename):
        # custom error message
        message = """
ERROR: Unable to locate file {}.

FIX: Have you tried running python build_data.py first?
This will build vocab file from your train, test and dev sets and
trimm your word vectors.
""".format(filename)
        super(MyIOError, self).__init__(message)


class CoNLLDataset(object):
    """Class that iterates over CoNLL Dataset

    __iter__ method yields a tuple (words, tags)
        words: list of raw words
        tags: list of raw tags

    If processing_word and processing_tag are not None,
    optional preprocessing is appplied

    Example:
        ```python
        data = CoNLLDataset(filename)
        for sentence, tags in data:
            pass
        ```

    """
    def __init__(self, filename, processing_word=None, processing_tag=None,
                 max_iter=None):
        """
        Args:
            filename: path to the file
            processing_words: (optional) function that takes a word as input
            processing_tags: (optional) function that takes a tag as input
            max_iter: (optional) max number of sentences to yield

        """
        self.filename = filename
        self.processing_word = processing_word
        self.processing_tag = processing_tag
        self.max_iter = max_iter
        self.length = None


    def __iter__(self):
        niter = 0
        n_line = 0
        #print('começando a ler o arquivo')
        with open(self.filename) as f:
            words, tags = [], []
            linhas = f.readlines()
            id_linha = -1
            for line in linhas:
                id_linha += 1
                n_line += 1
                #line = line.strip()
                if (len(line.split()) == 0 or line.startswith("-DOCSTART-") or line.endswith('.txt\n') or id_linha == len(linhas)-1):
                #if line.endswith('.txt\n') or id_linha == len(linhas)-1:
                    #print('line:', line)
                    if len(words) != 0:
                        niter += 1
                        if self.max_iter is not None and niter > self.max_iter:
                            break
                        #print('words', words)
                        #print('tags', tags)
                        yield words, tags
                        words, tags = [], []
                else:
                    ls = line.split()
                    if len(ls) == 2:
                      word, tag = ls[0], ls[-1]
                      if tag == ',':
                          print(n_line)
                      if self.processing_word is not None:
                          word = self.processing_word(word)
                      try:
                        if self.processing_tag is not None:
                            tag = self.processing_tag(tag)
                      except:
                        print('erro ao obter a tag', tag)
                      words += [word]
                      tags += [tag]


    def __len__(self):
        """Iterates once over the corpus to set and store length"""
        if self.length is None:
            self.length = 0
            for _ in self:
                self.length += 1

        return self.length


def get_vocabs(datasets):
    """Build vocabulary from an iterable of datasets objects

    Args:
        datasets: a list of dataset objects

    Returns:
        a set of all the words in the dataset

    """
    print("Building vocab...")
    vocab_words = set()
    vocab_tags = set()
    for dataset in datasets:
        for words, tags in dataset:
          #print('words:')
          #print(words)
          #print('tags')
          #print(tags)
          if words != '' and tags != '':
            vocab_words.update(words)
            vocab_tags.update(tags)
    print("- done. {} tokens".format(len(vocab_words)))
    return vocab_words, vocab_tags


def get_char_vocab(dataset):
    """Build char vocabulary from an iterable of datasets objects

    Args:
        dataset: a iterator yielding tuples (sentence, tags)

    Returns:
        a set of all the characters in the dataset

    """
    vocab_char = set()
    for words, _ in dataset:
        for word in words:
            vocab_char.update(word)

    return vocab_char


def get_glove_vocab(filename):
    """Load vocab from file

    Args:
        filename: path to the glove vectors

    Returns:
        vocab: set() of strings
    """
    print("Building vocab...")
    vocab = set()
    with open(filename) as f:
        for line in f:
            words = line.strip().split(' ')
            if (len(words) == 2):
              word = words[0]
              vocab.add(word)
    print("- done. {} tokens".format(len(vocab)))
    return vocab


def write_vocab(vocab, filename):
    """Writes a vocab to a file

    Writes one word per line.

    Args:
        vocab: iterable that yields word
        filename: path to vocab file

    Returns:
        write a word per line

    """
    print("Writing vocab...")
    with open(filename, "w") as f:
        for i, word in enumerate(vocab):
            if i != len(vocab) - 1:
                f.write("{}\n".format(word))
            else:
                f.write(word)
    print("- done. {} tokens".format(len(vocab)))


def load_vocab(filename):
    """Loads vocab from a file

    Args:
        filename: (string) the format of the file must be one word per line.

    Returns:
        d: dict[word] = index

    """
    try:
        d = dict()
        with open(filename) as f:
            for idx, word in enumerate(f):
                word = word.strip()
                d[word] = idx

    except IOError:
        raise MyIOError(filename)
    return d


def export_trimmed_glove_vectors(vocab, glove_filename, trimmed_filename, dim):
    """Saves glove vectors in numpy array

    Args:
        vocab: dictionary vocab[word] = index
        glove_filename: a path to a glove file
        trimmed_filename: a path where to store a matrix in npy
        dim: (int) dimension of embeddings

    """
    embeddings = np.zeros([len(vocab), dim])
    with open(glove_filename) as f:
        for line in f:
            line = line.strip().split(' ')
            word = line[0]
            embedding = [float(x) for x in line[1:]]
            if word in vocab:
                word_idx = vocab[word]
                embeddings[word_idx] = np.asarray(embedding)

    np.savez_compressed(trimmed_filename, embeddings=embeddings)


def get_trimmed_glove_vectors(filename):
    """
    Args:
        filename: path to the npz file

    Returns:
        matrix of embeddings (np array)

    """
    try:
        with np.load(filename) as data:
            return data["embeddings"]

    except IOError:
        raise MyIOError(filename)


def get_processing_word(vocab_words=None, vocab_chars=None,
                    lowercase=False, chars=False, allow_unk=True):
    """Return lambda function that transform a word (string) into list,
    or tuple of (list, id) of int corresponding to the ids of the word and
    its corresponding characters.

    Args:
        vocab: dict[word] = idx

    Returns:
        f("cat") = ([12, 4, 32], 12345)
                 = (list of char ids, word id)

    """
    def f(word):
        # 0. get chars of words
        if vocab_chars is not None and chars == True:
            char_ids = []
            for char in word:
                # ignore chars out of vocabulary
                if char in vocab_chars:
                    char_ids += [vocab_chars[char]]

        # 1. preprocess word
        if lowercase:
            word = word.lower()
        word = re.sub('\d', '0', word)

        # 2. get id of word
        if vocab_words is not None:
            if word in vocab_words:
                word = vocab_words[word]
            else:
                if word.isdigit():
                    word = vocab_words[NUM]
                elif allow_unk:
                    word = vocab_words[UNK]
                else:
                    print(word)
                    raise Exception("Unknow key is not allowed. Check that "\
                                    "your vocab (tags?) is correct")

        # 3. return tuple char ids, word id
        if vocab_chars is not None and chars == True:
            return char_ids, word
        else:
            return word

    return f


def _pad_sequences(sequences, pad_tok, max_length):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with

    Returns:
        a list of list where each sublist has same length
    """
    sequence_padded, sequence_length = [], []

    for seq in sequences:
        seq = list(seq)
        seq_ = seq[:max_length] + [pad_tok]*max(max_length - len(seq), 0)
        sequence_padded +=  [seq_]
        sequence_length += [min(len(seq), max_length)]

    return sequence_padded, sequence_length


def pad_sequences(sequences, pad_tok, nlevels=1):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with
        nlevels: "depth" of padding, for the case where we have characters ids

    Returns:
        a list of list where each sublist has same length

    """
    if nlevels == 1:
        max_length = max(map(lambda x : len(x), sequences))
        sequence_padded, sequence_length = _pad_sequences(sequences,
                                            pad_tok, max_length)

    elif nlevels == 2:
        max_length_word = max([max(map(lambda x: len(x), seq))
                               for seq in sequences])
        sequence_padded, sequence_length = [], []
        for seq in sequences:
            # all words are same length now
            sp, sl = _pad_sequences(seq, pad_tok, max_length_word)
            sequence_padded += [sp]
            sequence_length += [sl]

        max_length_sentence = max(map(lambda x : len(x), sequences))
        sequence_padded, _ = _pad_sequences(sequence_padded,
                [pad_tok]*max_length_word, max_length_sentence)
        sequence_length, _ = _pad_sequences(sequence_length, 0,
                max_length_sentence)

    return sequence_padded, sequence_length


def minibatches(data, minibatch_size):
    """
    Args:
        data: generator of (sentence, tags) tuples
        minibatch_size: (int)

    Yields:
        list of tuples

    """
    x_batch, y_batch = [], []
    for (x, y) in data:
        if len(x_batch) == minibatch_size:
            yield x_batch, y_batch
            x_batch, y_batch = [], []

        if type(x[0]) == tuple:
            x = zip(*x)
        x_batch += [x]
        y_batch += [y]

    if len(x_batch) != 0:
        yield x_batch, y_batch


def get_chunk_type(tok, idx_to_tag):
    """
    Args:
        tok: id of token, ex 4
        idx_to_tag: dictionary {4: "B-PER", ...}

    Returns:
        tuple: "B", "PER"

    """
    tag_name = idx_to_tag[tok]
    tag_class = tag_name.split('-')[0]
    tag_type = tag_name.split('-')[-1]
    return tag_class, tag_type


def get_chunks(seq, tags):
    """Given a sequence of tags, group entities and their position

    Args:
        seq: [4, 4, 0, 0, ...] sequence of labels
        tags: dict["O"] = 4

    Returns:
        list of (chunk_type, chunk_start, chunk_end)

    Example:
        seq = [4, 5, 0, 3]
        tags = {"B-PER": 4, "I-PER": 5, "B-LOC": 3}
        result = [("PER", 0, 2), ("LOC", 3, 4)]

    """
    default = tags[NONE]
    idx_to_tag = {idx: tag for tag, idx in tags.items()}
    chunks = []
    chunk_type, chunk_start = None, None
    for i, tok in enumerate(seq):
        # End of a chunk 1
        if tok == default and chunk_type is not None:
            # Add a chunk.
            chunk = (chunk_type, chunk_start, i)
            chunks.append(chunk)
            chunk_type, chunk_start = None, None

        # End of a chunk + start of a chunk!
        elif tok != default:
            tok_chunk_class, tok_chunk_type = get_chunk_type(tok, idx_to_tag)
            if chunk_type is None:
                chunk_type, chunk_start = tok_chunk_type, i
            elif tok_chunk_type != chunk_type or tok_chunk_class == "B":
                chunk = (chunk_type, chunk_start, i)
                chunks.append(chunk)
                chunk_type, chunk_start = tok_chunk_type, i
        else:
            pass

    # end condition
    if chunk_type is not None:
        chunk = (chunk_type, chunk_start, len(seq))
        chunks.append(chunk)

    return chunks

def create_tag_dict(filename):
    '''
    From tags create dictionary grouping same classes
    '''
    # index to tag dic
    indxToTag = dict()
    with open(filename) as tags:
        i = 0
        for line in tags:
            tag = line.rstrip('\n')
            #if tag[0:2] in ["B-", "I-"]:
            #    tag = tag[2:]
            indxToTag[i] = tag
            i +=1
    return indxToTag

In [ ]:
import time
import sys
import logging
import numpy as np


def get_logger(filename):
    """Return a logger instance that writes in filename

    Args:
        filename: (string) path to log.txt

    Returns:
        logger: (instance of logger)

    """
    logger = logging.getLogger('logger')
    logger.setLevel(logging.DEBUG)
    logging.basicConfig(format='%(message)s', level=logging.DEBUG)
    handler = logging.FileHandler(filename)
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(logging.Formatter(
            '%(asctime)s:%(levelname)s: %(message)s'))
    logging.getLogger().addHandler(handler)

    return logger


class Progbar(object):
    """Progbar class copied from keras (https://github.com/fchollet/keras/)

    Displays a progress bar.
    Small edit : added strict arg to update
    # Arguments
        target: Total number of steps expected.
        interval: Minimum visual progress update interval (in seconds).
    """

    def __init__(self, target, width=30, verbose=1):
        self.width = width
        self.target = target
        self.sum_values = {}
        self.unique_values = []
        self.start = time.time()
        self.total_width = 0
        self.seen_so_far = 0
        self.verbose = verbose

    def update(self, current, values=[], exact=[], strict=[]):
        """
        Updates the progress bar.
        # Arguments
            current: Index of current step.
            values: List of tuples (name, value_for_last_step).
                The progress bar will display averages for these values.
            exact: List of tuples (name, value_for_last_step).
                The progress bar will display these values directly.
        """

        for k, v in values:
            if k not in self.sum_values:
                self.sum_values[k] = [v * (current - self.seen_so_far),
                                      current - self.seen_so_far]
                self.unique_values.append(k)
            else:
                self.sum_values[k][0] += v * (current - self.seen_so_far)
                self.sum_values[k][1] += (current - self.seen_so_far)
        for k, v in exact:
            if k not in self.sum_values:
                self.unique_values.append(k)
            self.sum_values[k] = [v, 1]

        for k, v in strict:
            if k not in self.sum_values:
                self.unique_values.append(k)
            self.sum_values[k] = v

        self.seen_so_far = current

        now = time.time()
        if self.verbose == 1:
            prev_total_width = self.total_width
            sys.stdout.write("\b" * prev_total_width)
            sys.stdout.write("\r")

            numdigits = int(np.floor(np.log10(self.target))) + 1
            barstr = '%%%dd/%%%dd [' % (numdigits, numdigits)
            bar = barstr % (current, self.target)
            prog = float(current)/self.target
            prog_width = int(self.width*prog)
            if prog_width > 0:
                bar += ('='*(prog_width-1))
                if current < self.target:
                    bar += '>'
                else:
                    bar += '='
            bar += ('.'*(self.width-prog_width))
            bar += ']'
            sys.stdout.write(bar)
            self.total_width = len(bar)

            if current:
                time_per_unit = (now - self.start) / current
            else:
                time_per_unit = 0
            eta = time_per_unit*(self.target - current)
            info = ''
            if current < self.target:
                info += ' - ETA: %ds' % eta
            else:
                info += ' - %ds' % (now - self.start)
            for k in self.unique_values:
                if type(self.sum_values[k]) is list:
                    info += ' - %s: %.4f' % (k,
                        self.sum_values[k][0] / max(1, self.sum_values[k][1]))
                else:
                    info += ' - %s: %s' % (k, self.sum_values[k])

            self.total_width += len(info)
            if prev_total_width > self.total_width:
                info += ((prev_total_width-self.total_width) * " ")

            sys.stdout.write(info)
            sys.stdout.flush()

            if current >= self.target:
                sys.stdout.write("\n")

        if self.verbose == 2:
            if current >= self.target:
                info = '%ds' % (now - self.start)
                for k in self.unique_values:
                    info += ' - %s: %.4f' % (k,
                        self.sum_values[k][0] / max(1, self.sum_values[k][1]))
                sys.stdout.write(info + "\n")

    def add(self, n, values=[]):
        self.update(self.seen_so_far+n, values)

In [ ]:
pip install tensorflow==1.14.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 80 kB/s 
     |████████████████████████████████| 488 kB 61.8 MB/s 
     |████████████████████████████████| 3.1 MB 71.8 MB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [ ]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [ ]:
import os
import tensorflow as tf


class BaseModel(object):
    """Generic class for general methods that are not specific to NER"""

    def __init__(self, config):
        """Defines self.config and self.logger

        Args:
            config: (Config instance) class with hyper parameters,
                vocab and embeddings

        """
        self.config = config
        self.logger = config.logger
        self.sess   = None
        self.saver  = None


    def reinitialize_weights(self, scope_name):
        """Reinitializes the weights of a given layer"""
        variables = tf.contrib.framework.get_variables(scope_name)
        init = tf.variables_initializer(variables)
        self.sess.run(init)


    def add_train_op(self, lr_method, lr, loss, clip=-1):
        """Defines self.train_op that performs an update on a batch

        Args:
            lr_method: (string) sgd method, for example "adam"
            lr: (tf.placeholder) tf.float32, learning rate
            loss: (tensor) tf.float32 loss to minimize
            clip: (python float) clipping of gradient. If < 0, no clipping

        """
        _lr_m = lr_method.lower() # lower to make sure

        with tf.variable_scope("train_step"):
            if _lr_m == 'adam': # sgd method
                optimizer = tf.train.AdamOptimizer(lr)
            elif _lr_m == 'adagrad':
                optimizer = tf.train.AdagradOptimizer(lr)
            elif _lr_m == 'sgd':
                optimizer = tf.train.MomentumOptimizer(lr, 0.9)
            elif _lr_m == 'rmsprop':
                optimizer = tf.train.RMSPropOptimizer(lr)
            else:
                raise NotImplementedError("Unknown method {}".format(_lr_m))

            if clip > 0: # gradient clipping if clip is positive
                grads, vs     = zip(*optimizer.compute_gradients(loss))
                grads, gnorm  = tf.clip_by_global_norm(grads, clip)
                self.train_op = optimizer.apply_gradients(zip(grads, vs))
            else:
                self.train_op = optimizer.minimize(loss)


    def initialize_session(self):
        """Defines self.sess and initialize the variables"""
        self.logger.info("Initializing tf session")
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()


    def restore_session(self, dir_model):
        """Reload weights into session

        Args:
            sess: tf.Session()
            dir_model: dir with weights

        """
        self.logger.info("Reloading the latest trained model...")
        self.saver.restore(self.sess, dir_model)


    def save_session(self):
        """Saves session = weights"""
        if not os.path.exists(self.config.dir_model):
            os.makedirs(self.config.dir_model)
        self.saver.save(self.sess, self.config.dir_model)


    def close_session(self):
        """Closes the session"""
        self.sess.close()


    def add_summary(self):
        """Defines variables for Tensorboard

        Args:
            dir_output: (string) where the results are written

        """
        self.merged      = tf.summary.merge_all()
        self.file_writer = tf.summary.FileWriter(self.config.dir_output,
                self.sess.graph)


    def train(self, train, dev, best_score=0):
        """Performs training with early stopping and lr exponential decay

        Args:
            train: dataset that yields tuple of (sentences, tags)
            dev: dataset

        """
        nepoch_no_imprv = 0 # for early stopping
        self.add_summary() # tensorboard

        for epoch in range(self.config.nepochs):
            self.logger.info("Epoch {:} out of {:}".format(epoch + 1,
                        self.config.nepochs))

            score = self.run_epoch(train, dev, epoch)
            self.config.lr *= self.config.lr_decay # decay learning rate

            # early stopping and saving best parameters
            if score >= best_score:
                nepoch_no_imprv = 0
                self.save_session()
                best_score = score
                self.logger.info("- new best score!")
            else:
                nepoch_no_imprv += 1
                if nepoch_no_imprv >= self.config.nepoch_no_imprv:
                    self.logger.info("- early stopping {} epochs without "\
                            "improvement".format(nepoch_no_imprv))
                    break


    def evaluate(self, test):
        """Evaluate model on test set

        Args:
            test: instance of class Dataset

        """
        self.logger.info("Testing model over test set: ")
        metrics = self.run_evaluate(test)
        msg = " - ".join(["{} {:04.2f}".format(k, v)
                for k, v in metrics.items()])
        self.logger.info(msg)

In [ ]:
pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.5 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=7c26a142522fbe3648ae1e3f174c7a69eaf5b2a1ee1634e351b9e64b6eda7ee2
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
def level_entity_new (labels, preds):

  labels_cm, preds_cm = [], []

  for id, lab in enumerate(labels):
    encontrou = False
    #verifica a predição das entidades != de O
    if lab.startswith('B-'):
      encontrou = False
      id_inicio, id_fim = id, id

      #verifica onde termina a entidade
      for id_prox in range(id+1, len(labels)):
        if not labels[id_prox].startswith('I-'):
          id_fim = id_prox - 1
          break

      ent_pred = 'O'
      for id_pred in range(id_inicio, id_fim+1):
        if preds[id_pred][2:] == lab[2:]:
          ent_pred = preds[id_pred]
          encontrou = True
          break
        else:
          ent_pred = preds[id_pred]

      labels_cm.append(lab)
      if ent_pred == 'O':
        preds_cm.append('O')
      else:
        preds_cm.append('B-' + ent_pred[2:])
    elif lab == 'O':
      #verifica se a predição e a label são O ou se a predição é uma parte da label e já foi contada anteriormente
      if preds[id] == lab or encontrou:
        labels_cm.append('O')
        preds_cm.append('O')
      else:
        if preds[id].startswith('B-'):
          encontrou_pred = False
          #verifica até onde termina a entidade se terá uma label real
          for id_prox in range(id+1, len(labels)):
            if preds[id_prox].startswith('I-'):
              if labels[id_prox] != 'O':
                encontrou_pred = True
                break
            else:
              break
          #se não tiver label anotada em toda a entidade predita, adiciona;
          #caso contrário, não adiciona, pois será adicionado pela verificação da label
          if not encontrou_pred:
            labels_cm.append('O')
            preds_cm.append('B-' + preds[id][2:])

  cr = classification_report([labels_cm], [preds_cm])

  header = sorted(list(set(labels_cm + preds_cm)))
  cm = confusion_matrix(labels_cm, preds_cm, labels=header)
  mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(cm)]
  content_cm = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

  return cr, content_cm

In [ ]:
'''
Code modified by the authors of leNER-Br paper:
    modify run_evaluate to base score on token correctness
'''


import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import f1_score, accuracy_score
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix


class NERModel(BaseModel):
    """Specialized class of Model for NER"""

    def __init__(self, config):
        super(NERModel, self).__init__(config)
        self.idx_to_tag = {idx: tag for tag, idx in
                           self.config.vocab_tags.items()}


    def add_placeholders(self):
        """Define placeholders = entries to computational graph"""
        # shape = (batch size, max length of sentence in batch)
        self.word_ids = tf.placeholder(tf.int32, shape=[None, None],
                        name="word_ids")

        # shape = (batch size)
        self.sequence_lengths = tf.placeholder(tf.int32, shape=[None],
                        name="sequence_lengths")

        # shape = (batch size, max length of sentence, max length of word)
        self.char_ids = tf.placeholder(tf.int32, shape=[None, None, None],
                        name="char_ids")

        # shape = (batch_size, max_length of sentence)
        self.word_lengths = tf.placeholder(tf.int32, shape=[None, None],
                        name="word_lengths")

        # shape = (batch size, max length of sentence in batch)
        self.labels = tf.placeholder(tf.int32, shape=[None, None],
                        name="labels")

        # hyper parameters
        self.dropout = tf.placeholder(dtype=tf.float32, shape=[],
                        name="dropout")
        self.lr = tf.placeholder(dtype=tf.float32, shape=[],
                        name="lr")


    def get_feed_dict(self, words, labels=None, lr=None, dropout=None):
        """Given some data, pad it and build a feed dictionary

        Args:
            words: list of sentences. A sentence is a list of ids of a list of
                words. A word is a list of ids
            labels: list of ids
            lr: (float) learning rate
            dropout: (float) keep prob

        Returns:
            dict {placeholder: value}

        """
        # perform padding of the given data
        if self.config.use_chars:
            char_ids, word_ids = zip(*words)
            word_ids, sequence_lengths = pad_sequences(word_ids, 0)
            char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0,
                nlevels=2)
        else:
            word_ids, sequence_lengths = pad_sequences(words, 0)

        # build feed dictionary
        feed = {
            self.word_ids: word_ids,
            self.sequence_lengths: sequence_lengths
        }

        if self.config.use_chars:
            feed[self.char_ids] = char_ids
            feed[self.word_lengths] = word_lengths

        if labels is not None:
            labels, _ = pad_sequences(labels, 0)
            feed[self.labels] = labels

        if lr is not None:
            feed[self.lr] = lr

        if dropout is not None:
            feed[self.dropout] = dropout

        return feed, sequence_lengths


    def add_word_embeddings_op(self):
        """Defines self.word_embeddings

        If self.config.embeddings is not None and is a np array initialized
        with pre-trained word vectors, the word embeddings is just a look-up
        and we don't train the vectors. Otherwise, a random matrix with
        the correct shape is initialized.
        """
        with tf.variable_scope("words"):
            if self.config.embeddings is None:
                self.logger.info("WARNING: randomly initializing word vectors")
                _word_embeddings = tf.get_variable(
                        name="_word_embeddings",
                        dtype=tf.float32,
                        shape=[self.config.nwords, self.config.dim_word])
            else:
                _word_embeddings = tf.Variable(
                        self.config.embeddings,
                        name="_word_embeddings",
                        dtype=tf.float32,
                        trainable=self.config.train_embeddings)

            word_embeddings = tf.nn.embedding_lookup(_word_embeddings,
                    self.word_ids, name="word_embeddings")

        with tf.variable_scope("chars"):
            if self.config.use_chars:
                # get char embeddings matrix
                _char_embeddings = tf.get_variable(
                        name="_char_embeddings",
                        dtype=tf.float32,
                        shape=[self.config.nchars, self.config.dim_char])
                char_embeddings = tf.nn.embedding_lookup(_char_embeddings,
                        self.char_ids, name="char_embeddings")

                # put the time dimension on axis=1
                s = tf.shape(char_embeddings)
                char_embeddings = tf.reshape(char_embeddings,
                        shape=[s[0]*s[1], s[-2], self.config.dim_char])
                word_lengths = tf.reshape(self.word_lengths, shape=[s[0]*s[1]])

                # bi lstm on chars
                cell_fw = tf.contrib.rnn.LSTMCell(self.config.hidden_size_char,
                        state_is_tuple=True)
                cell_bw = tf.contrib.rnn.LSTMCell(self.config.hidden_size_char,
                        state_is_tuple=True)
                _output = tf.nn.bidirectional_dynamic_rnn(
                        cell_fw, cell_bw, char_embeddings,
                        sequence_length=word_lengths, dtype=tf.float32)

                # read and concat output
                _, ((_, output_fw), (_, output_bw)) = _output
                output = tf.concat([output_fw, output_bw], axis=-1)

                # shape = (batch size, max sentence length, char hidden size)
                output = tf.reshape(output,
                        shape=[s[0], s[1], 2*self.config.hidden_size_char])
                word_embeddings = tf.concat([word_embeddings, output], axis=-1)

        self.word_embeddings =  tf.nn.dropout(word_embeddings, self.dropout)


    def add_logits_op(self):
        """Defines self.logits

        For each word in each sentence of the batch, it corresponds to a vector
        of scores, of dimension equal to the number of tags.
        """
        with tf.variable_scope("bi-lstm"):
            cell_fw = tf.contrib.rnn.LSTMCell(self.config.hidden_size_lstm)
            cell_bw = tf.contrib.rnn.LSTMCell(self.config.hidden_size_lstm)
            (output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(
                    cell_fw, cell_bw, self.word_embeddings,
                    sequence_length=self.sequence_lengths, dtype=tf.float32)
            output = tf.concat([output_fw, output_bw], axis=-1)
            output = tf.nn.dropout(output, self.dropout)

        with tf.variable_scope("proj"):
            W = tf.get_variable("W", dtype=tf.float32,
                    shape=[2*self.config.hidden_size_lstm, self.config.ntags])

            b = tf.get_variable("b", shape=[self.config.ntags],
                    dtype=tf.float32, initializer=tf.zeros_initializer())

            nsteps = tf.shape(output)[1]
            output = tf.reshape(output, [-1, 2*self.config.hidden_size_lstm])
            pred = tf.matmul(output, W) + b
            self.logits = tf.reshape(pred, [-1, nsteps, self.config.ntags])


    def add_pred_op(self):
        """Defines self.labels_pred

        This op is defined only in the case where we don't use a CRF since in
        that case we can make the prediction "in the graph" (thanks to tf
        functions in other words). With theCRF, as the inference is coded
        in python and not in pure tensroflow, we have to make the prediciton
        outside the graph.
        """
        if not self.config.use_crf:
            self.labels_pred = tf.cast(tf.argmax(self.logits, axis=-1),
                    tf.int32)


    def add_loss_op(self):
        """Defines the loss"""
        if self.config.use_crf:
            log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(
                    self.logits, self.labels, self.sequence_lengths)
            self.trans_params = trans_params # need to evaluate it for decoding
            self.loss = tf.reduce_mean(-log_likelihood)
        else:
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=self.logits, labels=self.labels)
            mask = tf.sequence_mask(self.sequence_lengths)
            losses = tf.boolean_mask(losses, mask)
            self.loss = tf.reduce_mean(losses)

        # for tensorboard
        tf.summary.scalar("loss", self.loss)


    def build(self):
        # NER specific functions
        self.add_placeholders()
        self.add_word_embeddings_op()
        self.add_logits_op()
        self.add_pred_op()
        self.add_loss_op()

        # Generic functions that add training op and initialize session
        self.add_train_op(self.config.lr_method, self.lr, self.loss,
                self.config.clip)
        self.initialize_session() # now self.sess is defined and vars are init


    def predict_batch(self, words):
        """
        Args:
            words: list of sentences

        Returns:
            labels_pred: list of labels for each sentence
            sequence_length

        """
        fd, sequence_lengths = self.get_feed_dict(words, dropout=1.0)

        if self.config.use_crf:
            # get tag scores and transition params of CRF
            viterbi_sequences = []
            logits, trans_params = self.sess.run(
                    [self.logits, self.trans_params], feed_dict=fd)

            # iterate over the sentences because no batching in vitervi_decode
            for logit, sequence_length in zip(logits, sequence_lengths):
                logit = logit[:sequence_length] # keep only the valid steps
                viterbi_seq, viterbi_score = tf.contrib.crf.viterbi_decode(
                        logit, trans_params)
                viterbi_sequences += [viterbi_seq]

            return viterbi_sequences, sequence_lengths

        else:
            labels_pred = self.sess.run(self.labels_pred, feed_dict=fd)

            return labels_pred, sequence_lengths


    def run_epoch(self, train, dev, epoch):
        """Performs one complete pass over the train set and evaluate on dev

        Args:
            train: dataset that yields tuple of sentences, tags
            dev: dataset
            epoch: (int) index of the current epoch

        Returns:
            f1: (python float), score to select model on, higher is better

        """
        # progbar stuff for logging
        batch_size = self.config.batch_size
        nbatches = (len(train) + batch_size - 1) // batch_size
        prog = Progbar(target=nbatches)

        # iterate over dataset
        for i, (words, labels) in enumerate(minibatches(train, batch_size)):
            fd, _ = self.get_feed_dict(words, labels, self.config.lr,
                    self.config.dropout)

            _, train_loss, summary = self.sess.run(
                    [self.train_op, self.loss, self.merged], feed_dict=fd)

            prog.update(i + 1, [("train loss", train_loss)])

            # tensorboard
            if i % 10 == 0:
                self.file_writer.add_summary(summary, epoch*nbatches + i)

        metrics = self.run_evaluate(dev)
        msg = " - ".join(["{} {:04.2f}".format(k, v)
                for k, v in metrics.items()])
        self.logger.info(msg)

        return metrics["f1"]


    def run_evaluate(self, test):
        """Evaluates performance on test set

        Args:
            test: dataset that yields tuple of (sentences, tags)

        Returns:
            metrics: (dict) metrics["acc"] = 98.4, ...

        """
        preds = []
        labels = []

        # index to tag dic
        indxToTag = create_tag_dict("/content/drive/MyDrive/UNIFOR/NER/CDJur/results_lstm/tags.txt")

        for words, labs in minibatches(test, self.config.batch_size):
            labels_pred, sequence_lengths = self.predict_batch(words)

            for lab, lab_pred, length in zip(labs, labels_pred,
                                             sequence_lengths):
                lab_pred = lab_pred[:length]
                lab = lab[:length]
                preds.append(lab_pred)
                labels.append(lab)

        preds_t = [indxToTag.get(item, item) for items in preds for item in items]
        labels_t = [indxToTag.get(item, item) for items in labels for item in items]
        f1 = f1_score(labels_t, preds_t, average='micro', labels=['JURISPRUDENCIA', 'LOCAL', 'TEMPO',  'PESSOA', 'LEGISLACAO', 'ORGANIZACAO'])
        acc = accuracy_score(labels_t, preds_t)
        print('Labels:')
        print(labels_t)
        print('Preds:')
        print(preds_t)
        print('Classification_report:')
        print(classification_report([labels_t], [preds_t]))
        l_aux = labels_t
        p_aux = preds_t

        #Ajusta as transições de tags inválidas do padrão IOB2 (O I ; B-X I-Y; I-X I-Y)
        for idx_pred in range(len(preds_t)):
          if preds_t[idx_pred].startswith('I-') and \
              (idx_pred == 0 or (idx_pred > 0 and preds_t[idx_pred-1] == 'O')):
            #print('mudou: ' + toplevel_preds[0][idx_pred])
            preds_t[idx_pred] = 'B-' + preds_t[idx_pred][2:]
            #print('para: ' + toplevel_preds[0][idx_pred])
          elif idx_pred > 0 and preds_t[idx_pred].startswith('I-') and \
              (preds_t[idx_pred-1].startswith('I-') or preds_t[idx_pred-1].startswith('B-')) and \
               preds_t[idx_pred][2:] != preds_t[idx_pred-1][2:]:
            #print('mudou: ' + toplevel_preds[0][idx_pred])
            preds_t[idx_pred] = 'I-' + preds_t[idx_pred-1][2:]
            #print('para: ' + toplevel_preds[0][idx_pred]

        cr_level_entity, cm_level_entity = level_entity_new(labels_t, preds_t)
        print(f"Classification Report Level Entity:\n {cr_level_entity}")
        print(cm_level_entity)

        #matrix_confusion_class(labels, preds)
        #print(confusion_matrix([labels],[preds]))

        return {"acc": 100*acc, "f1": 100*f1}

    def matrix_confusion_class(labels, preds):
      labels_true_cm, labels_pred_cm = [], []
      for l in labels:
        labels_true_cm.extend(l)

      for l in preds:
        labels_pred_cm.extend(l)

      header = sorted(list(set(labels_true_cm + labels_pred_cm)))
      matrix = confusion_matrix(labels_true_cm, labels_pred_cm, labels=header)
      print(header)
      print(matrix)

    #def matrix_confusion_class(preds, labels):
    #  preds_flat = np.argmax(preds, axis=1).flatten()
    #  labels_flat = labels.flatten()
    #  print(confusion_matrix(labels_flat,preds_flat))

    def predict(self, words_raw):
        """Returns list of tags

        Args:
            words_raw: list of words (string), just one sentence (no batch)

        Returns:
            preds: list of tags (string), one for each word in the sentence

        """
        words = [self.config.processing_word(w) for w in words_raw]
        if type(words[0]) == tuple:
            words = zip(*words)
        pred_ids, _ = self.predict_batch([words])
        preds = [self.idx_to_tag[idx] for idx in list(pred_ids[0])]

        return preds


In [ ]:
l_aux = []
p_aux = []

In [ ]:
'''
Configs set by the authors of leNER-Br paper
'''

import os


class Config():
    def __init__(self, load=True):
        """Initialize hyperparameters and load vocabs

        Args:
            load_embeddings: (bool) if True, load embeddings into
                np array, else None

        """
        # directory for training outputs
        if not os.path.exists(self.dir_output):
            os.makedirs(self.dir_output)

        # create instance of logger
        self.logger = get_logger(self.path_log)

        # load if requested (default)
        if load:
            self.load()


    def load(self):
        """Loads vocabulary, processing functions and embeddings

        Supposes that build_data.py has been run successfully and that
        the corresponding files have been created (vocab and trimmed GloVe
        vectors)

        """
        # 1. vocabulary
        self.vocab_words = load_vocab(self.filename_words)
        self.vocab_tags  = load_vocab(self.filename_tags)
        self.vocab_chars = load_vocab(self.filename_chars)

        self.nwords     = len(self.vocab_words)
        self.nchars     = len(self.vocab_chars)
        self.ntags      = len(self.vocab_tags)

        # 2. get processing functions that map str -> id
        self.processing_word = get_processing_word(self.vocab_words,
                self.vocab_chars, lowercase=True, chars=self.use_chars)
        self.processing_tag  = get_processing_word(self.vocab_tags,
                lowercase=False, allow_unk=False)

        # 3. get pre-trained embeddings
        self.embeddings = (get_trimmed_glove_vectors(self.filename_trimmed)
                if self.use_pretrained else None)


    # general config
    dir_output = "/content/drive/MyDrive/UNIFOR/NER/CDJur/results_lstm/prototype_revised/"
    dir_model  = dir_output + "model.weights/"
    path_log   = dir_output + "log.txt"

    # embeddings
    dim_word = 300
    dim_char = 50

    # glove files
    filename_glove = "/content/drive/MyDrive/UNIFOR/NER/CDJur/lener/glove_s300.txt" #enviar para jorge - ok
    # trimmed embeddings (created from glove_filename with build_data.py)
    filename_trimmed = "/content/drive/MyDrive/UNIFOR/NER/CDJur/results_lstm/glove.6B.{}d.trimmed.npz".format(dim_word)
    use_pretrained = True

    # dataset
    # filename_dev = "data/coNLL/eng/eng.testa.iob"
    # filename_test = "data/coNLL/eng/eng.testb.iob"
    # filename_train = "data/coNLL/eng/eng.train.iob"

    #filename_dev = '/content/drive/MyDrive/Unifor/NER/lener/dev.txt'
    #filename_test = '/content/drive/MyDrive/Unifor/NER/lener/test.txt'
    #filename_train = '/content/drive/MyDrive/Unifor/NER/lener/train.txt'


    #entidade = 'exp1/'
    #path_geral = '/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_210922/' + entidade
    #filename_dev = f'{path_geral}val.conll'
    #filename_test = f'{path_geral}teste.conll'
    #filename_train = f'{path_geral}treino.conll'

    data_revisao = '131022'
    filename_train = f'/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_{data_revisao}/exp1/treino.conll'
    filename_dev = f'/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_{data_revisao}/exp1/val.conll'
    filename_test = f'/content/drive/MyDrive/UNIFOR/NER/CDJur/cdjur_revisao_{data_revisao}/exp1/teste.conll'


    #filename_train = '/content/drive/MyDrive/Unifor/NER/lener/train_atualizado.txt'

    max_iter = None # if not None, max number of examples in Dataset

    # vocab (created from dataset with build_data.py)
    filename_words = "/content/drive/MyDrive/UNIFOR/NER/CDJur/results_lstm/words.txt"
    filename_tags = "/content/drive/MyDrive/UNIFOR/NER/CDJur/results_lstm/tags.txt"
    filename_chars = "/content/drive/MyDrive/UNIFOR/NER/CDJur/results_lstm/chars.txt"

    # training
    train_embeddings = True
    nepochs          = 20#55#1#25#55
    dropout          = 0.5
    batch_size       = 10
    lr_method        = "sgd"
    lr               = 0.015
    lr_decay         = 0.95
    clip             = 5 # if negative, no clipping
    nepoch_no_imprv  = 20

    # model hyperparameters
    hidden_size_char = 25 # lstm on chars
    hidden_size_lstm = 100 # lstm on word embeddings

    # NOTE: if both chars and crf, only 1.6x slower on GPU
    use_crf = True # if crf, training is 1.7x slower on CPU
    use_chars = True # if char embedding, training is 3.5x slower on CPU


In [ ]:
config = Config(load=False)
processing_word = get_processing_word(lowercase=True)

# Generators
print('train')
train = CoNLLDataset(config.filename_train, processing_word)
print('dev')
dev   = CoNLLDataset(config.filename_dev, processing_word)
print('test')
test  = CoNLLDataset(config.filename_test, processing_word)

train
dev
test


In [ ]:
# Build Word and Tag vocab
# vocab_words, vocab_tags = get_vocabs([train, dev, test, test2])
vocab_words, vocab_tags = get_vocabs([train, dev])
vocab_glove = get_glove_vocab(config.filename_glove)

vocab = vocab_words & vocab_glove
vocab.add(UNK)
vocab.add(NUM)

# Save vocab
write_vocab(vocab, config.filename_words)
write_vocab(vocab_tags, config.filename_tags)

Building vocab...
- done. 69735 tokens
Building vocab...
- done. 1 tokens
Writing vocab...
- done. 2 tokens
Writing vocab...
- done. 43 tokens


In [ ]:
# Trim GloVe Vectors
vocab = load_vocab(config.filename_words)
export_trimmed_glove_vectors(vocab, config.filename_glove,
                            config.filename_trimmed, config.dim_word)

# Build and save char vocab
train = CoNLLDataset(config.filename_train)
vocab_chars = get_char_vocab(train)
write_vocab(vocab_chars, config.filename_chars)

Writing vocab...
- done. 145 tokens


In [ ]:
config = Config()

# build model
model = NERModel(config)
model.build()
score = 0
# model.restore_session(config.dir_model) # optional, restore weights
# model.reinitialize_weights("proj")
# make sure to make score equals models best score

# create datasets
dev   = CoNLLDataset(config.filename_dev, config.processing_word, config.processing_tag, config.max_iter)
train = CoNLLDataset(config.filename_train, config.processing_word, config.processing_tag, config.max_iter)
test = CoNLLDataset(config.filename_test, config.processing_word, config.processing_tag, config.max_iter)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, whi

INFO:logger:Initializing tf session


In [ ]:
model.train(train, dev, score)

INFO:logger:Epoch 1 out of 20


2291/2291 [==============================] - 1122s - train loss: 9.5235  


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

           END_AUTOR       0.00      0.00      0.00        21
          END_DELITO       0.00      0.00      0.00        81
          END_OUTROS       0.00      0.00      0.00       124
             END_REU       0.00      0.00      0.00       155
      END_TESTEMUNHA       0.00      0.00      0.00        78
          END_VITIMA       0.00      0.00      0.00        21
       NOR_ACESSORIA       0.52      0.56      0.54      1151
  NOR_JURISPRUDÊNCIA       0.00      0.00      0.00       318
       NOR_PRINCIPAL       0.35      0.13      0.19      1145
                PENA       0.00      0.00      0.00        70
           PES_ADVOG       0.00      0.00      0.00       155
           PES_AUTOR       0.00      0.00      0.00       222
PES_AUTORID_POLICIAL       0.00      0.00      0.00       531
            PES_JUIZ       0.00      0.00      0.00       125
          PES_OUTROS       0.09      0.00      0.00      1312
     PE

INFO:logger:acc 90.40 - f1 0.00


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



                      precision    recall  f1-score   support

           END_AUTOR       0.90      0.43      0.58        21
          END_DELITO       0.69      0.25      0.36        81
          END_OUTROS       0.07      0.01      0.01       124
             END_REU       0.27      0.66      0.38       155
      END_TESTEMUNHA       0.68      0.32      0.43        78
          END_VITIMA       0.09      0.05      0.06        21
       NOR_ACESSORIA       0.60      0.77      0.68      1151
  NOR_JURISPRUDÊNCIA       0.74      0.77      0.75       318
       NOR_PRINCIPAL       0.79      0.56      0.65      1145
                PENA       0.39      0.39      0.39        70
           PES_ADVOG       0.74      0.47      0.57       155
           PES_AUTOR       0.69      0.30      0.42       222
PES_AUTORID_POLICIAL       0.74      0.48      0.58       531
            PES_JUIZ       0.66      0.58      0.62       125
          PES_OUTROS       0.46      0.38      0.42      1312
     PE

INFO:logger:acc 80.94 - f1 0.00


Classification Report Level Entity:
                       precision    recall  f1-score   support

           END_AUTOR       1.00      0.43      0.60        21
          END_DELITO       0.76      0.27      0.40        81
          END_OUTROS       0.42      0.04      0.07       124
             END_REU       0.34      0.93      0.50       155
      END_TESTEMUNHA       0.81      0.37      0.51        78
          END_VITIMA       0.09      0.05      0.06        21
       NOR_ACESSORIA       0.68      0.89      0.77      1151
  NOR_JURISPRUDÊNCIA       0.89      0.91      0.90       318
       NOR_PRINCIPAL       0.87      0.62      0.72      1145
                PENA       0.55      0.60      0.58        70
           PES_ADVOG       0.91      0.62      0.74       155
           PES_AUTOR       0.73      0.32      0.45       222
PES_AUTORID_POLICIAL       0.87      0.56      0.68       531
            PES_JUIZ       0.81      0.72      0.76       125
          PES_OUTROS       0.55 

INFO:logger:- new best score!


In [ ]:
#test  = CoNLLDataset('/content/drive/MyDrive/Unifor/NER/cdjur/cdjur_v8/test/labeled/test.conll', processing_word)

model.evaluate(test)

INFO:logger:Testing model over test set: 
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

           END_AUTOR       0.47      0.23      0.31        40
          END_DELITO       0.57      0.21      0.30       156
          END_OUTROS       0.09      0.02      0.03       153
             END_REU       0.23      0.61      0.33       283
      END_TESTEMUNHA       0.65      0.34      0.45       120
          END_VITIMA       0.19      0.07      0.11        69
       NOR_ACESSORIA       0.59      0.79      0.68      2365
  NOR_JURISPRUDÊNCIA       0.71      0.76      0.74       753
       NOR_PRINCIPAL       0.74      0.51      0.60      2747
                PENA       0.54      0.45      0.49       139
           PES_ADVOG       0.52      0.23      0.32       228
           PES_AUTOR       0.74      0.33      0.45       481
PES_AUTORID_POLICIAL       0.80      0.73      0.76       822
            PES_JUIZ       0.66      0.57      0.61       183
          PES_OUTROS       0.55      0.40      0.46      2652
     PE

INFO:logger:acc 83.93 - f1 0.00


Classification Report Level Entity:
                       precision    recall  f1-score   support

           END_AUTOR       0.53      0.23      0.32        40
          END_DELITO       0.86      0.31      0.45       156
          END_OUTROS       0.12      0.03      0.04       153
             END_REU       0.32      0.89      0.47       283
      END_TESTEMUNHA       0.66      0.34      0.45       120
          END_VITIMA       0.29      0.14      0.19        69
       NOR_ACESSORIA       0.67      0.90      0.77      2365
  NOR_JURISPRUDÊNCIA       0.86      0.92      0.89       753
       NOR_PRINCIPAL       0.91      0.63      0.75      2747
                PENA       0.68      0.60      0.64       139
           PES_ADVOG       0.77      0.35      0.48       228
           PES_AUTOR       0.79      0.35      0.49       481
PES_AUTORID_POLICIAL       0.90      0.82      0.86       822
            PES_JUIZ       0.74      0.64      0.69       183
          PES_OUTROS       0.65 